In [1]:
# All Imports

import numpy as np
import numba
from numba import njit, prange
from numba import cuda
import copy
import sys
import time
import matplotlib.pyplot as plt
import pydicom
import glob
from skimage import filters
from skimage.filters import unsharp_mask


In [ ]:
#host_prj_allangle_backup   = load_prj_std(0) # Load prelog  data
#host_scat_allangle_backup  = load_prj_std(1) # Load scatter data
#host_blank_allangle_backup = load_prj_std(2) # Load blank   data

In [2]:
# Settings Array
# CE01
# BINSx   = 3584
# BINSy   = 2816
# BINSy   = 1000

# Vsize_x = 0.085
# Vsize_y = 0.085
# Vsize_z = 1

# x_p0    = -93.5
# y_p0    = -115.0
# z_p0    = -30.0

# x_d0    = -152.32
# y_d0    = -119

In [2]:
# Ray Tracing Code


@njit(parallel=True)
def x_y_flip(host_prjbuf_temp):
    host_prjbuf_temp_gpu  = np.copy(host_prjbuf_temp)
    host_prjbuf_1view_gpu = np.copy(host_prjbuf_temp)
    
    for i in prange(BINSx):
        for j in prange(BINSy):
            bin_ind_temp = j*BINSx+i
            bin_ind      = i*BINSy+j
            host_prjbuf_1view_gpu[bin_ind] = host_prjbuf_temp_gpu[bin_ind_temp]
    
    return host_prjbuf_1view_gpu

@njit(parallel=True)
def compute_yry(host_prj_allangle, host_scat_allangle):
    all_b_size        =  ANGLES*BINSx*BINSy
    host_yry_allangle =  np.zeros(BINSx*BINSy*ANGLES)
    
    for i in prange(all_b_size):
        if (host_prj_allangle[i] == 0):
            host_yry_allangle[i] = 0
        else:
            dif = host_prj_allangle[i] - host_scat_allangle[i]
            if (dif <= 0):
                dif = host_prj_allangle[i]
            host_yry_allangle[i] = (dif*dif)/host_prj_allangle[i]
    
    return host_yry_allangle

@njit(parallel=True)
def compute_gamma_yry(host_yry_allangle, host_gamma_allangle):
    all_b_size              = ANGLES*BINSx*BINSy
    host_gamma_yry_allangle = np.zeros(all_b_size)
    
    for i in prange(all_b_size):
        host_gamma_yry_allangle[i] = host_yry_allangle[i]*host_gamma_allangle[i]
    
    return host_gamma_yry_allangle

@njit(parallel=True)
def compute_h(host_prj_sub, host_blank_sub, host_line_sub, host_scat_sub):
    ANGLES_per_sub  = int(ANGLES/subset_num)
    sub_b_size      = int(ANGLES_per_sub*BINSx*BINSy)
    
    host_sub = np.zeros(sub_b_size)
    
    for i in prange(sub_b_size):
        y_tmp       = host_blank_sub[i]*np.exp(-host_line_sub[i])
        host_sub[i] = (host_prj_sub[i]/(y_tmp+host_scat_sub[i])-1)*y_tmp
    
    return host_sub

@njit(parallel=True)
def update_est(host_est, host_capL, host_RD, host_d, host_RDD):
    f_size    = IMGSIZx*IMGSIZy*IMGSIZz
    host_est1 = np.zeros(f_size)
    
    for i in prange(f_size):
        host_est1[i] = host_est[i]-(host_capL[i]+beta*host_RD[i])/(host_d[i]+2*beta*host_RDD[i])
        if (host_est1[i] < 0):
            host_est1[i] = 0
    
    return host_est1

@njit(parallel=True)
def regroup_prj(host_uponregroup_allangle):
    all_b_size     = int(ANGLES*BINSx*BINSy)
    ANGLES_per_sub = int(ANGLES/subset_num)
    b_size         = int(BINSx*BINSy)
    
    host_allangle_tmp = np.zeros(host_uponregroup_allangle.shape)
    flag              = 0
    
    for i in range(subset_num):
        for j in range(ANGLES_per_sub):
            for k in range(b_size):
                host_allangle_tmp[flag] = host_uponregroup_allangle[int((j*subset_num+i)*b_size+k)]
                flag = flag +1
    
    return host_allangle_tmp

import math

@cuda.jit(device=True)
def rayTrace3D_GPU_direct_notexturememory_normprj(d_normprj, x0, y0, z0, x1, y1, z1, status, sum_norm, bin_ind):
    # Perform Ray Tracing
    sum_norm = 0
    dx     = x1-x0
    dy     = y1-y0
    dz     = z1-z0
    Length = math.sqrt( dx*dx+dy*dy+dz*dz )


    if (x1 != x0):
        min_lx = (x_p0-x0)/dx
        max_lx = min_lx+IMGSIZx*Vsize_x/dx

        if (min_lx > max_lx):
            #SWAP(min_lx, max_lx);
            s_temp = min_lx
            min_lx = max_lx
            max_lx = s_temp
    else:
        # the line perpendicular to x axis
        if (x0 >= IMGSIZx*Vsize_x+x_p0 or x0<=x_p0):
            status = -1
            return
        min_lx = -1e3
        max_lx = 1e3
    
    
    if (y0 != y1):
        min_ly = (y_p0-y0)/dy
        max_ly = min_ly+IMGSIZy*Vsize_y/dy

        if (min_ly > max_ly):
            #SWAP(min_ly, max_ly);
            s_temp = min_ly
            min_ly = max_ly
            max_ly = s_temp
    else:
        # the line perpendicular to y axis
        if (y0 >= IMGSIZy*Vsize_y+y_p0 or y0 <= y_p0):
            status = -1
            return
        min_ly = -1e3
        max_ly = 1e3

    
    if (z0 != z1):
        min_lz = (z_p0-z0)/dz
        max_lz = min_lz+IMGSIZz*Vsize_z/dz
        if (min_lz > max_lz):
            #SWAP(min_lz, max_lz);
            s_temp = min_lz
            min_lz = max_lz
            max_lz = s_temp
    else:
        # the line perpendicular to z axis
        if (z0 >= IMGSIZz*Vsize_z+z_p0 or z0 <= z_p0):
            status = -1
            return
        min_lz = -1e3
        max_lz = 1e3
    
    
    max_l = max_lx
    if (max_l > max_ly):
        max_l=max_ly
    if (max_l > max_lz):
        max_l = max_lz

    min_l = min_lx
    if (min_l < min_ly):
        min_l = min_ly
    if (min_l < min_lz):
        min_l = min_lz

    if (min_l >= max_l):
        status = -1
        return
    
    if (min_lx != min_l):
        prev_x = (int)(math.floor( (min_l* dx + x0 - x_p0) / Vsize_x ))

        if (x0 < x1):
            min_lx= ((prev_x+1)*Vsize_x+x_p0-x0)/ dx
        else:
            if (x0 == x1):
                min_lx = 1e3
            else:
                min_lx = (prev_x*Vsize_x+x_p0-x0) / dx
    else:
        if (x0 < x1):
            prev_x = 0
            min_lx = ( Vsize_x+x_p0-x0 )/ dx
        else:
            prev_x = IMGSIZx-1
            min_lx = ( prev_x*Vsize_x+x_p0-x0 )/ dx
    
    if (min_ly != min_l):
        prev_y = (int)(math.floor( (min_l* dy + y0 - y_p0)/Vsize_y ))
        if (y0 < y1):
            min_ly = ( (prev_y+1)*Vsize_y+y_p0-y0)/ dy
        else:
            if (y0==y1):
                min_ly = 1e3
            else:
                min_ly = (prev_y*Vsize_y+y_p0-y0)/ dy
    else:
        if (y0<y1):
            prev_y = 0
            min_ly = ( Vsize_y+y_p0-y0 )/ dy
        else:
            prev_y = IMGSIZy-1
            min_ly = ( prev_y*Vsize_y+y_p0-y0 )/ dy
    
    if (min_lz != min_l):
        prev_z = (int)(math.floor( (min_l* dz + z0 - z_p0)/Vsize_z ))
        if (z0 < z1):
            min_lz = ( (prev_z+1)*Vsize_z+z_p0-z0)/ dz
        else:
            if (z0 == z1):
                min_lz = 1e3
            else:
                min_lz = (prev_z*Vsize_z+z_p0-z0)/ dz
    else:
        if (z0 < z1):
            prev_z = 0
            min_lz = ( Vsize_z+z_p0-z0 )/ dz
        else:
            prev_z = (int)(IMGSIZz-1)
            min_lz = ( prev_z*Vsize_z+z_p0-z0 )/dz
    
    
    min_l_new = min_lx
    if (min_l_new > min_ly):
        min_l_new = min_ly
    if (min_l_new > min_lz):
        min_l_new = min_lz
    
    incx = Vsize_x/dx
    incy = Vsize_y/dy
    incz = Vsize_z/dz

    ind = 0
    
    while ( (max_l-min_l_new)/max_l > 0.000001):
        tmp_length = (min_l_new-min_l)*Length; #<-a_ij
        if ((prev_x >= 0) and (prev_x < IMGSIZx) and (prev_y >= 0) and (prev_y < IMGSIZy) and (prev_z >= 0) and (prev_z < IMGSIZz)):
            sum_norm = sum_norm + 1*tmp_length
        
        ind = ind + 1
        if (min_l_new == min_lx):
            if (x0 < x1):
                prev_x = prev_x + 1
                min_lx = min_lx + incx; #Vsize_x/dx
            else:
                prev_x = prev_x - 1
                min_lx = min_lx - incx; #Vsize_x/dx;
        else:
            prev_x = prev_x


        if (min_l_new == min_ly):
            if (y0 < y1):
                prev_y = prev_y + 1
                min_ly = min_ly + incy; #Vsize_y / dy;
            else:
                prev_y = prev_y - 1
                min_ly = min_ly- incy; #Vsize_y/dy;
        else:
            prev_y = prev_y


        if (min_l_new == min_lz):
            if (z0 < z1):
                prev_z = prev_z + 1
                min_lz = min_lz + incz #Vsize_z/dz;
            else:
                prev_z = prev_z - 1
                min_lz = min_lz - incz; #Vsize_z/dz
        else:
            prev_z = prev_z

        min_l     = min_l_new
        min_l_new = min_lx

        if (min_l_new > min_ly):
            min_l_new = min_ly

        if (min_l_new>min_lz):
            min_l_new=min_lz
        
        
        tmp_length = (max_l-min_l)*Length
        if ((prev_x>=0) and (prev_x<IMGSIZx) and (prev_y>=0) and (prev_y<IMGSIZy) and (prev_z>=0) and (prev_z<IMGSIZz)):
            sum_norm = sum_norm + 1*tmp_length
        
        d_normprj[bin_ind] = sum_norm

@cuda.jit(device=True)
def rayTrace3D_GPU_direct_notexturememory(d_normprj, d_prjbuf, d_objbuf, x0, y0, z0, x1, y1, z1, status):
    ix, iy   = cuda.grid(2)
    
    status   = 0
    #sum_norm = 0
    
    for a in range(angleStart, angleEnd):
        s         = d_index[a]
        theta     = d_angles[s]
        sin_theta = math.sin(theta)
        cos_theta = math.cos(theta)
        x0        = sourceR*sin_theta
        z0        = sourceR*cos_theta
        y0        = sourceY
        
        # calculate bin index
        i = nbBinsX*((int)(BINSx/nBatchBINSx)) + ix
        j = nbBinsY*((int)(BINSy/nBatchBINSy)) + iy

        bin_x_pos = (x_d0+(i+0.5)*Bsize_x)
        bin_y_pos = (y_d0+(j+0.5)*Bsize_y)

        x1 =  bin_x_pos
        z1 = -detectorR
        y1 =  bin_y_pos

        # Iso-centric version
        # x1 =  bin_x_pos*cos_theta-detectorR*sin_theta
        # z1 = -bin_x_pos*sin_theta-detectorR*cos_theta
        # y1 =  bin_y_pos

        bin_ind = ((a-angleStart)*BINSx+i)*BINSy+j
        
        y0 = sourceY
        
        # Perform Ray Tracing
        fsum_norm = 0.0
        fsum      = 0.0
        
        dx     = x1-x0
        dy     = y1-y0
        dz     = z1-z0
        Length = math.sqrt( dx*dx + dy*dy + dz*dz )
        
        d_normprj[bin_ind] = 0
        d_prjbuf[bin_ind]  = 0
        
        if (x1 != x0):
            min_lx = (x_p0 - x0)/dx
            max_lx = min_lx + (IMGSIZx*Vsize_x)/dx
            if (min_lx > max_lx):
                #SWAP(min_lx, max_lx);
                s_temp = min_lx
                min_lx = max_lx
                max_lx = s_temp
        else:
            # the line perpendicular to x axis
            if ((x0 >= IMGSIZx*Vsize_x+x_p0) or x0 <= x_p0):
                status = -1
            min_lx = -1000.0
            max_lx =  1000.0
        
        if (y0 != y1):
            min_ly = (y_p0-y0)/dy
            max_ly = min_ly + IMGSIZy*Vsize_y/dy
            if (min_ly > max_ly):
                #SWAP(min_ly, max_ly);
                s_temp = min_ly
                min_ly = max_ly
                max_ly = s_temp
        else:
            # the line perpendicular to y axis
            if (y0 >= IMGSIZy*Vsize_y + y_p0 or y0 <= y_p0):
                status = -1
            min_ly = -1000.0
            max_ly =  1000.0
        
        if (z0 != z1):
            min_lz = (z_p0 - z0)/dz
            max_lz = min_lz + IMGSIZz*Vsize_z/dz
            if (min_lz > max_lz):
                #SWAP(min_lz, max_lz);
                s_temp = min_lz
                min_lz = max_lz
                max_lz = s_temp
        else:
            # the line perpendicular to z axis
            if (z0 >= IMGSIZz*Vsize_z+z_p0 or z0 <= z_p0):
                status = -1
            min_lz = -1000.0
            max_lz =  1000.0
        
        max_l = max_lx
        if (max_l > max_ly):
            max_l = max_ly
        if (max_l > max_lz):
            max_l = max_lz
        
        min_l = min_lx
        if (min_l < min_ly):
            min_l = min_ly
        if (min_l < min_lz):
            min_l = min_lz
        
        if (min_l >= max_l):
            status1 = 10
            #d_normprj[bin_ind] = 1
        else:
            status1 = 0
        if status1 != 10:
            if (min_lx != min_l):
                prev_x = (int)(math.floor( (min_l* dx + x0 - x_p0) / Vsize_x ))
                if (x0 < x1):
                    min_lx = ((prev_x+1)*Vsize_x+x_p0 - x0)/ dx
                elif (x0 == x1):
                    min_lx = 1000
                else:
                    min_lx = (prev_x*Vsize_x+x_p0-x0) / dx
                #d_normprj[bin_ind] = Vsize_x
            else:
                if (x0 < x1):
                    prev_x = 0
                    min_lx = ( Vsize_x+x_p0-x0 )/ dx
                else:
                    prev_x = IMGSIZx-1
                    min_lx = ( prev_x*Vsize_x + x_p0 - x0 )/ dx
            #d_normprj[bin_ind] = prev_x
                
            if (min_ly != min_l):
                prev_y = (int)(math.floor( (min_l* dy + y0 - y_p0)/ Vsize_y ))
                if (y0 < y1):
                    min_ly = ( (prev_y+1)*Vsize_y + y_p0 - y0)/ dy
                elif (y0 == y1):
                    min_ly = 1000
                else:
                    min_ly = (prev_y*Vsize_y + y_p0 - y0)/ dy
            else:
                if (y0 < y1):
                    prev_y = 0
                    min_ly = ( Vsize_y+y_p0-y0 )/ dy
                else:
                    prev_y = IMGSIZy-1
                    min_ly = ( prev_y*Vsize_y + y_p0 - y0 )/ dy
                
            if (min_lz != min_l):
                prev_z = (int)(math.floor( (min_l* dz + z0 - z_p0)/ Vsize_z ))
                if (z0 < z1):
                    min_lz = ( (prev_z+1)*Vsize_z+z_p0-z0)/ dz
                elif (z0 == z1):
                    min_lz = 1000
                else:
                    min_lz = (prev_z*Vsize_z + z_p0 - z0)/ dz
            else:
                if (z0 < z1):
                    prev_z = 0
                    min_lz = ( Vsize_z + z_p0 - z0 )/ dz
                else:
                    prev_z = (int)(IMGSIZz-1)
                    min_lz = ( prev_z*Vsize_z+z_p0-z0 )/dz
            
            min_l_new = min_lx
            if (min_l_new > min_ly):
                min_l_new = min_ly
            if (min_l_new > min_lz):
                min_l_new = min_lz
            
            incx = Vsize_x/dx
            incy = Vsize_y/dy
            incz = Vsize_z/dz
            
            ind = 0
            #d_normprj[bin_ind] = max_l
            while ( (max_l-min_l_new)/max_l > 0.000001):
                tmp_length = (min_l_new - min_l)*Length
                if ((prev_x >= 0) and (prev_x < IMGSIZx) and (prev_y >= 0) and (prev_y < IMGSIZy) and (prev_z >= 0) and (prev_z < IMGSIZz)):
                    fsum_norm      = fsum_norm + 1*tmp_length
                    fsum           = fsum + d_objbuf[(prev_z*IMGSIZy+prev_y)*IMGSIZx+prev_x]*tmp_length
                
                ind = ind + 1
                if (min_l_new == min_lx):
                    if (x0 < x1):
                        prev_x = prev_x + 1
                        min_lx = min_lx + incx #Vsize_x/dx
                    else:
                        prev_x = prev_x - 1
                        min_lx = min_lx - incx #Vsize_x/dx;
                else:
                    prev_x = prev_x

                if (min_l_new == min_ly):
                    if (y0 < y1):
                        prev_y = prev_y + 1
                        min_ly = min_ly + incy #Vsize_y / dy;
                    else:
                        prev_y = prev_y - 1
                        min_ly = min_ly- incy #Vsize_y/dy;
                else:
                    prev_y = prev_y

                if (min_l_new == min_lz):
                    if (z0 < z1):
                        prev_z = prev_z + 1
                        min_lz = min_lz + incz #Vsize_z/dz;
                    else:
                        prev_z = prev_z - 1
                        min_lz = min_lz - incz; #Vsize_z/dz
                else:
                    prev_z = prev_z

                min_l     = min_l_new
                min_l_new = min_lx

                if (min_l_new > min_ly):
                    min_l_new = min_ly

                if (min_l_new > min_lz):
                    min_l_new = min_lz
            
            tmp_length = (max_l - min_l)*Length
            if ((prev_x >= 0) and (prev_x < IMGSIZx) and (prev_y >= 0) and (prev_y < IMGSIZy) and (prev_z >= 0) and (prev_z < IMGSIZz)):
                fsum_norm      = fsum_norm + 1*tmp_length
                fsum           = fsum + d_objbuf[(prev_z*IMGSIZy+prev_y)*IMGSIZx+prev_x]*tmp_length
            status2 = 100
        
        if status2 == 100:
            d_normprj[bin_ind] = fsum_norm
            d_prjbuf[bin_ind]  = fsum
        
        cuda.syncthreads()
    
@cuda.jit
def ray_trace_gpu_manyangles_direct_notexturememory_normprj(d_normprj, d_angles, d_index, angleStart, angleEnd, nbBinsX, nbBinsY):
    ix, iy   = cuda.grid(2)
    
    status   = 0
    #sum_norm = 0
    
    for a in range(angleStart, angleEnd):
        #print(a)
        s         = d_index[a]
        theta     = d_angles[s]
        sin_theta = math.sin(theta)
        cos_theta = math.cos(theta)
        x0        = sourceR*sin_theta
        z0        = sourceR*cos_theta
        y0        = sourceY

        # calculate bin index
        i = nbBinsX*((int)(BINSx/nBatchBINSx)) + ix
        j = nbBinsY*((int)(BINSy/nBatchBINSy)) + iy

        bin_x_pos = (x_d0+(i+0.5)*Bsize_x)
        bin_y_pos = (y_d0+(j+0.5)*Bsize_y)

        x1 =  bin_x_pos
        z1 = -detectorR
        y1 =  bin_y_pos

        # Iso-centric version
        # x1 =  bin_x_pos*cos_theta-detectorR*sin_theta
        # z1 = -bin_x_pos*sin_theta-detectorR*cos_theta
        # y1 =  bin_y_pos

        bin_ind = ((a-angleStart)*BINSx+i)*BINSy+j
        
        y0 = sourceY
        
        # Perform Ray Tracing
        sum_norm = 0.0
        dx     = x1-x0
        dy     = y1-y0
        dz     = z1-z0
        Length = math.sqrt( dx*dx + dy*dy + dz*dz )
        d_normprj[bin_ind] = 0
        
        if (x1 != x0):
            min_lx = (x_p0 - x0)/dx
            max_lx = min_lx + (IMGSIZx*Vsize_x)/dx
            if (min_lx > max_lx):
                #SWAP(min_lx, max_lx);
                s_temp = min_lx
                min_lx = max_lx
                max_lx = s_temp
        else:
            # the line perpendicular to x axis
            if ((x0 >= IMGSIZx*Vsize_x+x_p0) or x0 <= x_p0):
                status = -1
            min_lx = -1000.0
            max_lx =  1000.0
        
        if (y0 != y1):
            min_ly = (y_p0-y0)/dy
            max_ly = min_ly + IMGSIZy*Vsize_y/dy
            if (min_ly > max_ly):
                #SWAP(min_ly, max_ly);
                s_temp = min_ly
                min_ly = max_ly
                max_ly = s_temp
        else:
            # the line perpendicular to y axis
            if (y0 >= IMGSIZy*Vsize_y + y_p0 or y0 <= y_p0):
                status = -1
            min_ly = -1000.0
            max_ly =  1000.0
        
        if (z0 != z1):
            min_lz = (z_p0 - z0)/dz
            max_lz = min_lz + IMGSIZz*Vsize_z/dz
            if (min_lz > max_lz):
                #SWAP(min_lz, max_lz);
                s_temp = min_lz
                min_lz = max_lz
                max_lz = s_temp
        else:
            # the line perpendicular to z axis
            if (z0 >= IMGSIZz*Vsize_z+z_p0 or z0 <= z_p0):
                status = -1
            min_lz = -1000.0
            max_lz =  1000.0
        
        max_l = max_lx
        if (max_l > max_ly):
            max_l = max_ly
        if (max_l > max_lz):
            max_l = max_lz
        
        min_l = min_lx
        if (min_l < min_ly):
            min_l = min_ly
        if (min_l < min_lz):
            min_l = min_lz
        
        if (min_l >= max_l):
            status1 = 10
            #d_normprj[bin_ind] = 1
        else:
            status1 = 0
        if status1 != 10:
            if (min_lx != min_l):
                prev_x = (int)(math.floor( (min_l* dx + x0 - x_p0) / Vsize_x ))
                if (x0 < x1):
                    min_lx = ((prev_x+1)*Vsize_x+x_p0 - x0)/ dx
                elif (x0 == x1):
                    min_lx = 1000
                else:
                    min_lx = (prev_x*Vsize_x+x_p0-x0) / dx
                #d_normprj[bin_ind] = Vsize_x
            else:
                if (x0 < x1):
                    prev_x = 0
                    min_lx = ( Vsize_x+x_p0-x0 )/ dx
                else:
                    prev_x = IMGSIZx-1
                    min_lx = ( prev_x*Vsize_x + x_p0 - x0 )/ dx
            #d_normprj[bin_ind] = prev_x
                
            if (min_ly != min_l):
                prev_y = (int)(math.floor( (min_l* dy + y0 - y_p0)/ Vsize_y ))
                if (y0 < y1):
                    min_ly = ( (prev_y+1)*Vsize_y + y_p0 - y0)/ dy
                elif (y0 == y1):
                    min_ly = 1000
                else:
                    min_ly = (prev_y*Vsize_y + y_p0 - y0)/ dy
            else:
                if (y0 < y1):
                    prev_y = 0
                    min_ly = ( Vsize_y+y_p0-y0 )/ dy
                else:
                    prev_y = IMGSIZy-1
                    min_ly = ( prev_y*Vsize_y + y_p0 - y0 )/ dy
                
            if (min_lz != min_l):
                prev_z = (int)(math.floor( (min_l* dz + z0 - z_p0)/ Vsize_z ))
                if (z0 < z1):
                    min_lz = ( (prev_z+1)*Vsize_z+z_p0-z0)/ dz
                elif (z0 == z1):
                    min_lz = 1000
                else:
                    min_lz = (prev_z*Vsize_z + z_p0 - z0)/ dz
            else:
                if (z0 < z1):
                    prev_z = 0
                    min_lz = ( Vsize_z + z_p0 - z0 )/ dz
                else:
                    prev_z = (int)(IMGSIZz-1)
                    min_lz = ( prev_z*Vsize_z+z_p0-z0 )/dz
            
            min_l_new = min_lx
            if (min_l_new > min_ly):
                min_l_new = min_ly
            if (min_l_new > min_lz):
                min_l_new = min_lz


            incx = Vsize_x/dx
            incy = Vsize_y/dy
            incz = Vsize_z/dz

            ind = 0
            #d_normprj[bin_ind] = max_l
            while ( (max_l-min_l_new)/max_l > 0.000001):
                tmp_length = (min_l_new - min_l)*Length
                if ((prev_x >= 0) and (prev_x < IMGSIZx) and (prev_y >= 0) and (prev_y < IMGSIZy) and (prev_z >= 0) and (prev_z < IMGSIZz)):
                    sum_norm = sum_norm + 1*tmp_length

                ind = ind + 1
                if (min_l_new == min_lx):
                    if (x0 < x1):
                        prev_x = prev_x + 1
                        min_lx = min_lx + incx #Vsize_x/dx
                    else:
                        prev_x = prev_x - 1
                        min_lx = min_lx - incx #Vsize_x/dx;
                else:
                    prev_x = prev_x

                if (min_l_new == min_ly):
                    if (y0 < y1):
                        prev_y = prev_y + 1
                        min_ly = min_ly + incy #Vsize_y / dy;
                    else:
                        prev_y = prev_y - 1
                        min_ly = min_ly- incy #Vsize_y/dy;
                else:
                    prev_y = prev_y

                if (min_l_new == min_lz):
                    if (z0 < z1):
                        prev_z = prev_z + 1
                        min_lz = min_lz + incz #Vsize_z/dz;
                    else:
                        prev_z = prev_z - 1
                        min_lz = min_lz - incz; #Vsize_z/dz
                else:
                    prev_z = prev_z

                min_l     = min_l_new
                min_l_new = min_lx

                if (min_l_new > min_ly):
                    min_l_new = min_ly

                if (min_l_new > min_lz):
                    min_l_new = min_lz
            
            tmp_length = (max_l - min_l)*Length
            if ((prev_x >= 0) and (prev_x < IMGSIZx) and (prev_y >= 0) and (prev_y < IMGSIZy) and (prev_z >= 0) and (prev_z < IMGSIZz)):
                sum_norm = sum_norm + 1*tmp_length
            status2 = 100
        if status2 == 100:
            d_normprj[bin_ind] = sum_norm
        #else:
        #    d_normprj[bin_ind] = sum_norm
#         elif status == 10:
#             d_normprj[bin_ind] = 100000
#         elif status == -1:
#             d_normprj[bin_ind] = 50000
#         else:
#             d_normprj[bin_ind] = 200000
#         d_normprj[bin_ind] = Length
        cuda.syncthreads()

@cuda.jit
def ray_trace_gpu_manyangles_direct_notexturememory_OSTR_cos(d_objbuf, d_prjbuf, d_angles, d_index, angleStart, angleEnd, nbBinsX, nbBinsY):
    ix, iy   = cuda.grid(2)
    
    status   = 0
    
    for a in range(angleStart, angleEnd):
        #print(a)
        s         = d_index[a]
        theta     = d_angles[s]
        sin_theta = math.sin(theta)
        cos_theta = math.cos(theta)
        x0        = sourceR*sin_theta
        z0        = sourceR*cos_theta
        y0        = sourceY
        
        # calculate bin index
        i = nbBinsX*((int)(BINSx/nBatchBINSx)) + ix
        j = nbBinsY*((int)(BINSy/nBatchBINSy)) + iy

        bin_x_pos = (x_d0+(i+0.5)*Bsize_x)
        bin_y_pos = (y_d0+(j+0.5)*Bsize_y)

        x1 =  bin_x_pos
        z1 = -detectorR
        y1 =  bin_y_pos

        # Iso-centric version
        # x1 =  bin_x_pos*cos_theta-detectorR*sin_theta
        # z1 = -bin_x_pos*sin_theta-detectorR*cos_theta
        # y1 =  bin_y_pos

        bin_ind = ((a-angleStart)*BINSx+i)*BINSy+j
        
        y0 = sourceY
        
        # Perform Ray Tracing
        sum_norm = 0.0
        dx     = x1-x0
        dy     = y1-y0
        dz     = z1-z0
        Length = math.sqrt( dx*dx + dy*dy + dz*dz )
        #d_prjbuf[bin_ind] = 0
        
        if (x1 != x0):
            min_lx = (x_p0 - x0)/dx
            max_lx = min_lx + (IMGSIZx*Vsize_x)/dx
            if (min_lx > max_lx):
                #SWAP(min_lx, max_lx);
                s_temp = min_lx
                min_lx = max_lx
                max_lx = s_temp
        else:
            # the line perpendicular to x axis
            if ((x0 >= IMGSIZx*Vsize_x+x_p0) or x0 <= x_p0):
                status = -1
            min_lx = -1000.0
            max_lx =  1000.0
        
        if (y0 != y1):
            min_ly = (y_p0-y0)/dy
            max_ly = min_ly + IMGSIZy*Vsize_y/dy
            if (min_ly > max_ly):
                #SWAP(min_ly, max_ly);
                s_temp = min_ly
                min_ly = max_ly
                max_ly = s_temp
        else:
            # the line perpendicular to y axis
            if (y0 >= IMGSIZy*Vsize_y + y_p0 or y0 <= y_p0):
                status = -1
            min_ly = -1000.0
            max_ly =  1000.0
        
        if (z0 != z1):
            min_lz = (z_p0 - z0)/dz
            max_lz = min_lz + IMGSIZz*Vsize_z/dz
            if (min_lz > max_lz):
                #SWAP(min_lz, max_lz);
                s_temp = min_lz
                min_lz = max_lz
                max_lz = s_temp
        else:
            # the line perpendicular to z axis
            if (z0 >= IMGSIZz*Vsize_z+z_p0 or z0 <= z_p0):
                status = -1
            min_lz = -1000.0
            max_lz =  1000.0
        
        max_l = max_lx
        if (max_l > max_ly):
            max_l = max_ly
        if (max_l > max_lz):
            max_l = max_lz
        
        min_l = min_lx
        if (min_l < min_ly):
            min_l = min_ly
        if (min_l < min_lz):
            min_l = min_lz
        
        if (min_l >= max_l):
            status1 = 10
            #d_normprj[bin_ind] = 1
        else:
            status1 = 0
        if status1 != 10:
            if (min_lx != min_l):
                prev_x = (int)(math.floor( (min_l* dx + x0 - x_p0) / Vsize_x ))
                if (x0 < x1):
                    min_lx = ((prev_x+1)*Vsize_x+x_p0 - x0)/ dx
                elif (x0 == x1):
                    min_lx = 1000
                else:
                    min_lx = (prev_x*Vsize_x+x_p0-x0) / dx
                #d_normprj[bin_ind] = Vsize_x
            else:
                if (x0 < x1):
                    prev_x = 0
                    min_lx = ( Vsize_x+x_p0-x0 )/ dx
                else:
                    prev_x = IMGSIZx-1
                    min_lx = ( prev_x*Vsize_x + x_p0 - x0 )/ dx
            #d_normprj[bin_ind] = prev_x
                
            if (min_ly != min_l):
                prev_y = (int)(math.floor( (min_l* dy + y0 - y_p0)/ Vsize_y ))
                if (y0 < y1):
                    min_ly = ( (prev_y+1)*Vsize_y + y_p0 - y0)/ dy
                elif (y0 == y1):
                    min_ly = 1000
                else:
                    min_ly = (prev_y*Vsize_y + y_p0 - y0)/ dy
            else:
                if (y0 < y1):
                    prev_y = 0
                    min_ly = ( Vsize_y+y_p0-y0 )/ dy
                else:
                    prev_y = IMGSIZy-1
                    min_ly = ( prev_y*Vsize_y + y_p0 - y0 )/ dy
                
            if (min_lz != min_l):
                prev_z = (int)(math.floor( (min_l* dz + z0 - z_p0)/ Vsize_z ))
                if (z0 < z1):
                    min_lz = ( (prev_z+1)*Vsize_z+z_p0-z0)/ dz
                elif (z0 == z1):
                    min_lz = 1000
                else:
                    min_lz = (prev_z*Vsize_z + z_p0 - z0)/ dz
            else:
                if (z0 < z1):
                    prev_z = 0
                    min_lz = ( Vsize_z + z_p0 - z0 )/ dz
                else:
                    prev_z = (int)(IMGSIZz-1)
                    min_lz = ( prev_z*Vsize_z+z_p0-z0 )/dz
            
            min_l_new = min_lx
            if (min_l_new > min_ly):
                min_l_new = min_ly
            if (min_l_new > min_lz):
                min_l_new = min_lz


            incx = Vsize_x/dx
            incy = Vsize_y/dy
            incz = Vsize_z/dz

            ind = 0
            #d_normprj[bin_ind] = max_l
            while ( (max_l-min_l_new)/max_l > 0.000001):
                tmp_length = (min_l_new - min_l)*Length
                if ((prev_x >= 0) and (prev_x < IMGSIZx) and (prev_y >= 0) and (prev_y < IMGSIZy) and (prev_z >= 0) and (prev_z < IMGSIZz)):
                    sum_norm = sum_norm + d_objbuf[(prev_z*IMGSIZy+prev_y)*IMGSIZx+prev_x]*tmp_length

                ind = ind + 1
                if (min_l_new == min_lx):
                    if (x0 < x1):
                        prev_x = prev_x + 1
                        min_lx = min_lx + incx #Vsize_x/dx
                    else:
                        prev_x = prev_x - 1
                        min_lx = min_lx - incx #Vsize_x/dx;
                else:
                    prev_x = prev_x

                if (min_l_new == min_ly):
                    if (y0 < y1):
                        prev_y = prev_y + 1
                        min_ly = min_ly + incy #Vsize_y / dy;
                    else:
                        prev_y = prev_y - 1
                        min_ly = min_ly- incy #Vsize_y/dy;
                else:
                    prev_y = prev_y

                if (min_l_new == min_lz):
                    if (z0 < z1):
                        prev_z = prev_z + 1
                        min_lz = min_lz + incz #Vsize_z/dz;
                    else:
                        prev_z = prev_z - 1
                        min_lz = min_lz - incz; #Vsize_z/dz
                else:
                    prev_z = prev_z

                min_l     = min_l_new
                min_l_new = min_lx

                if (min_l_new > min_ly):
                    min_l_new = min_ly

                if (min_l_new > min_lz):
                    min_l_new = min_lz
            
            tmp_length = (max_l - min_l)*Length
            if ((prev_x >= 0) and (prev_x < IMGSIZx) and (prev_y >= 0) and (prev_y < IMGSIZy) and (prev_z >= 0) and (prev_z < IMGSIZz)):
                sum_norm = sum_norm + d_objbuf[(prev_z*IMGSIZy+prev_y)*IMGSIZx+prev_x]*tmp_length
            status2 = 100
        
        if status2 == 100:
            d_prjbuf[bin_ind] = sum_norm*cos_theta
        cuda.syncthreads()

@cuda.jit
def ray_trace_gpu_manyangles_direct_notexturememory_cos(d_objbuf, d_prjbuf, d_normprj, d_angles, d_index, angleStart, angleEnd, nbBinsX, nbBinsY):
    ix, iy   = cuda.grid(2)
    status   = 0
    
    for a in range(angleStart, angleEnd):
        #print(a)
        s         = d_index[a]
        theta     = d_angles[s]
        sin_theta = math.sin(theta)
        cos_theta = math.cos(theta)
        x0        = sourceR*sin_theta
        z0        = sourceR*cos_theta
        y0        = sourceY
        
        # calculate bin index
        i = nbBinsX*((int)(BINSx/nBatchBINSx)) + ix
        j = nbBinsY*((int)(BINSy/nBatchBINSy)) + iy

        bin_x_pos = (x_d0+(i+0.5)*Bsize_x)
        bin_y_pos = (y_d0+(j+0.5)*Bsize_y)

        x1 =  bin_x_pos
        z1 = -detectorR
        y1 =  bin_y_pos

        # Iso-centric version
        # x1 =  bin_x_pos*cos_theta-detectorR*sin_theta
        # z1 = -bin_x_pos*sin_theta-detectorR*cos_theta
        # y1 =  bin_y_pos

        bin_ind = ((a-angleStart)*BINSx+i)*BINSy+j
        
        y0 = sourceY
        
        # Perform Ray Tracing
        fsum_norm = 0.0
        fsum      = 0.0
        
        dx     = x1-x0
        dy     = y1-y0
        dz     = z1-z0
        Length = math.sqrt( dx*dx + dy*dy + dz*dz )
        #d_prjbuf[bin_ind] = 0
        
        if (x1 != x0):
            min_lx = (x_p0 - x0)/dx
            max_lx = min_lx + (IMGSIZx*Vsize_x)/dx
            if (min_lx > max_lx):
                #SWAP(min_lx, max_lx);
                s_temp = min_lx
                min_lx = max_lx
                max_lx = s_temp
        else:
            # the line perpendicular to x axis
            if ((x0 >= IMGSIZx*Vsize_x+x_p0) or x0 <= x_p0):
                status = -1
            min_lx = -1000.0
            max_lx =  1000.0
        
        if (y0 != y1):
            min_ly = (y_p0-y0)/dy
            max_ly = min_ly + IMGSIZy*Vsize_y/dy
            if (min_ly > max_ly):
                #SWAP(min_ly, max_ly);
                s_temp = min_ly
                min_ly = max_ly
                max_ly = s_temp
        else:
            # the line perpendicular to y axis
            if (y0 >= IMGSIZy*Vsize_y + y_p0 or y0 <= y_p0):
                status = -1
            min_ly = -1000.0
            max_ly =  1000.0
        
        if (z0 != z1):
            min_lz = (z_p0 - z0)/dz
            max_lz = min_lz + IMGSIZz*Vsize_z/dz
            if (min_lz > max_lz):
                #SWAP(min_lz, max_lz);
                s_temp = min_lz
                min_lz = max_lz
                max_lz = s_temp
        else:
            # the line perpendicular to z axis
            if (z0 >= IMGSIZz*Vsize_z+z_p0 or z0 <= z_p0):
                status = -1
            min_lz = -1000.0
            max_lz =  1000.0
        
        max_l = max_lx
        if (max_l > max_ly):
            max_l = max_ly
        if (max_l > max_lz):
            max_l = max_lz
        
        min_l = min_lx
        if (min_l < min_ly):
            min_l = min_ly
        if (min_l < min_lz):
            min_l = min_lz
        
        if (min_l >= max_l):
            status1 = 10
            #d_normprj[bin_ind] = 1
        else:
            status1 = 0
        
        if status1 != 10:
            if (min_lx != min_l):
                prev_x = (int)(math.floor( (min_l* dx + x0 - x_p0) / Vsize_x ))
                if (x0 < x1):
                    min_lx = ((prev_x+1)*Vsize_x+x_p0 - x0)/ dx
                elif (x0 == x1):
                    min_lx = 1000
                else:
                    min_lx = (prev_x*Vsize_x+x_p0-x0) / dx
                #d_normprj[bin_ind] = Vsize_x
            else:
                if (x0 < x1):
                    prev_x = 0
                    min_lx = ( Vsize_x+x_p0-x0 )/ dx
                else:
                    prev_x = IMGSIZx-1
                    min_lx = ( prev_x*Vsize_x + x_p0 - x0 )/ dx
            #d_normprj[bin_ind] = prev_x
                
            if (min_ly != min_l):
                prev_y = (int)(math.floor( (min_l* dy + y0 - y_p0)/ Vsize_y ))
                if (y0 < y1):
                    min_ly = ( (prev_y+1)*Vsize_y + y_p0 - y0)/ dy
                elif (y0 == y1):
                    min_ly = 1000
                else:
                    min_ly = (prev_y*Vsize_y + y_p0 - y0)/ dy
            else:
                if (y0 < y1):
                    prev_y = 0
                    min_ly = ( Vsize_y+y_p0-y0 )/ dy
                else:
                    prev_y = IMGSIZy-1
                    min_ly = ( prev_y*Vsize_y + y_p0 - y0 )/ dy
                
            if (min_lz != min_l):
                prev_z = (int)(math.floor( (min_l* dz + z0 - z_p0)/ Vsize_z ))
                if (z0 < z1):
                    min_lz = ( (prev_z+1)*Vsize_z+z_p0-z0)/ dz
                elif (z0 == z1):
                    min_lz = 1000
                else:
                    min_lz = (prev_z*Vsize_z + z_p0 - z0)/ dz
            else:
                if (z0 < z1):
                    prev_z = 0
                    min_lz = ( Vsize_z + z_p0 - z0 )/ dz
                else:
                    prev_z = (int)(IMGSIZz-1)
                    min_lz = ( prev_z*Vsize_z+z_p0-z0 )/dz
            
            min_l_new = min_lx
            if (min_l_new > min_ly):
                min_l_new = min_ly
            if (min_l_new > min_lz):
                min_l_new = min_lz


            incx = Vsize_x/dx
            incy = Vsize_y/dy
            incz = Vsize_z/dz

            ind = 0
            #d_normprj[bin_ind] = max_l
            while ( (max_l-min_l_new)/max_l > 0.000001):
                tmp_length = (min_l_new - min_l)*Length
                if ((prev_x >= 0) and (prev_x < IMGSIZx) and (prev_y >= 0) and (prev_y < IMGSIZy) and (prev_z >= 0) and (prev_z < IMGSIZz)):
                    fsum_norm      = fsum_norm + 1*tmp_length
                    fsum           = fsum + d_objbuf[(prev_z*IMGSIZy+prev_y)*IMGSIZx+prev_x]*tmp_length

                ind = ind + 1
                if (min_l_new == min_lx):
                    if (x0 < x1):
                        prev_x = prev_x + 1
                        min_lx = min_lx + incx #Vsize_x/dx
                    else:
                        prev_x = prev_x - 1
                        min_lx = min_lx - incx #Vsize_x/dx;
                else:
                    prev_x = prev_x

                if (min_l_new == min_ly):
                    if (y0 < y1):
                        prev_y = prev_y + 1
                        min_ly = min_ly + incy #Vsize_y / dy;
                    else:
                        prev_y = prev_y - 1
                        min_ly = min_ly- incy #Vsize_y/dy;
                else:
                    prev_y = prev_y

                if (min_l_new == min_lz):
                    if (z0 < z1):
                        prev_z = prev_z + 1
                        min_lz = min_lz + incz #Vsize_z/dz;
                    else:
                        prev_z = prev_z - 1
                        min_lz = min_lz - incz; #Vsize_z/dz
                else:
                    prev_z = prev_z

                min_l     = min_l_new
                min_l_new = min_lx

                if (min_l_new > min_ly):
                    min_l_new = min_ly

                if (min_l_new > min_lz):
                    min_l_new = min_lz
            
            tmp_length = (max_l - min_l)*Length
            if ((prev_x >= 0) and (prev_x < IMGSIZx) and (prev_y >= 0) and (prev_y < IMGSIZy) and (prev_z >= 0) and (prev_z < IMGSIZz)):
                fsum_norm      = fsum_norm + 1*tmp_length
                fsum           = fsum + d_objbuf[(prev_z*IMGSIZy+prev_y)*IMGSIZx+prev_x]*tmp_length
            
            status2 = 100
        if status2 == 100:
            d_normprj[bin_ind] = fsum_norm
            d_prjbuf[bin_ind]  = fsum*cos_theta
        
        cuda.syncthreads()
    
def fprojectCB_1R_GPU_OSTR_normprj(d_normprj, d_angles, d_index, angleStart, angleEnd):
    PRJ_THREAD = PRJ_ThreX, PRJ_ThreY
    PRJ_GRID   = PRJ_GridX, PRJ_GridY
    
    for nbBinsX in range(nBatchBINSx):
        for nbBinsY in range(nBatchBINSy):
            ray_trace_gpu_manyangles_direct_notexturememory_normprj[PRJ_GRID, PRJ_THREAD](d_normprj, d_angles, d_index, angleStart, angleEnd, nbBinsX, nbBinsY)
            #cuda.synchronize()
    return

def fprojectCB_1R_GPU_OSTR_cos(estbuf, prj_est, d_angles, d_index, angleStart, angleEnd):
    PRJ_THREAD = PRJ_ThreX, PRJ_ThreY
    PRJ_GRID   = PRJ_GridX, PRJ_GridY
    
    for nbBinsX in range(nBatchBINSx):
        for nbBinsY in range(nBatchBINSy):
            ray_trace_gpu_manyangles_direct_notexturememory_OSTR_cos[PRJ_GRID, PRJ_THREAD](estbuf, prj_est, d_angles, d_index, 
                                                                                           angleStart, angleEnd, nbBinsX, nbBinsY)
            cuda.synchronize()
    return

def fprojectCB_1R_GPU_SART_cos(estbuf, prj_est, d_normprj, d_angles, d_index, angleStart, angleEnd):
    PRJ_THREAD = PRJ_ThreX, PRJ_ThreY
    PRJ_GRID   = PRJ_GridX, PRJ_GridY
    
    for nbBinsX in range(nBatchBINSx):
        for nbBinsY in range(nBatchBINSy):
            ray_trace_gpu_manyangles_direct_notexturememory_cos[PRJ_GRID, PRJ_THREAD](estbuf, prj_est, d_normprj, d_angles, 
                                                                                      d_index, angleStart, angleEnd, 
                                                                                      nbBinsX, nbBinsY)
            cuda.synchronize()
    return

@cuda.jit
def SART_prj_diff_kernel(diff_line, prjbuf, prj_est, normprj, d_index, angleStart, angleEnd, nbBinsX, nbBinsY):
    ix, iy   = cuda.grid(2)
    
    # calculate bin index
    for a in range(angleStart, angleEnd):
        i = nbBinsX*((int)(BINSx/nBatchBINSx)) + ix
        j = nbBinsY*((int)(BINSy/nBatchBINSy)) + iy
        
        bin_ind = ((a-angleStart)*BINSx+i)*BINSy+j
        
        if normprj[bin_ind] != 0:
            diff_line[bin_ind] = prjbuf[bin_ind]#bin_ind#(prjbuf[bin_ind] - prj_est[bin_ind])/normprj[bin_ind]
        else:
            diff_line[bin_ind] = bin_ind#1
    return

@njit(parallel=True)
def SART_prj_diff(diff_line, prjbuf, prj_est, normprj, d_index, angleStart, angleEnd):
    result_diff_line = np.zeros(prjbuf.shape)
    
    #for nbBinsX in prange(nBatchBINSx):
    #    for nbBinsY in prange(nBatchBINSy):
    for bin_ind in prange(prjbuf.shape[0]):
        if normprj[bin_ind] != 0:
            result_diff_line[bin_ind] = (prjbuf[bin_ind] - prj_est[bin_ind])/normprj[bin_ind]
        else:
            result_diff_line[bin_ind] = 0.0#prjbuf[bin_ind]#bin_ind
    
    return result_diff_line

#@njit(parallel=True)
def SART_prj_diff_old(diff_line, prjbuf, prj_est, normprj, d_index, angleStart, angleEnd):
    PRJ_THREAD = PRJ_ThreX, PRJ_ThreY
    PRJ_GRID   = PRJ_GridX, PRJ_GridY
    
    for nbBinsX in range(nBatchBINSx):
        for nbBinsY in range(nBatchBINSy):
            SART_prj_diff_kernel[PRJ_GRID, PRJ_THREAD](diff_line, prjbuf, prj_est, normprj, d_index, angleStart, angleEnd, nbBinsX, nbBinsY)
            cuda.synchronize()
    return
#SART_prj_diff_kernel<<<PRJ_GRID,PRJ_THREAD>>>(diff_line,prjbuf,prj_est,normprj,d_index,angleStart,angleEnd,nbBinsX,nbBinsY);
#CUT_CHECK_ERROR("Kernel execution failed");
#cudaThreadSynchronize()

#def SART_prj_diff(diff_line, prjbuf, prj_est, normprj, d_index, angleStart, angleEnd):

#@cuda.jit
def bprojectCB_4B_GPU_R_SART(d_objbuf, d_prjbuf, d_prior, d_index, 
                                   d_angles, angleStart, angleEnd, lambda_parameter, beta):
    BACKPRJ_THREAD = BACKPRJ_ThreX, BACKPRJ_ThreY
    BACKPRJ_GRID   = BACKPRJ_GridX, BACKPRJ_GridY
    
    for nbatchIDx in range(nBatchXdim):
        backprj_OSSART_gpu_manyviews_R[BACKPRJ_GRID, BACKPRJ_THREAD](d_objbuf, d_prjbuf, d_prior, d_index, 
                                                                            d_angles, angleStart, angleEnd , nbatchIDx, lambda_parameter, beta)
        cuda.synchronize()
    return

@cuda.jit
def backprj_OSSART_gpu_manyviews_R(d_objbuf, d_prjbuf, d_prior, d_index, d_angles, 
                                          angleStart, angleEnd, nbatchIDx, lambda_parameter, beta):
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y
    
    tx = cuda.threadIdx.x + (nbatchIDx*cuda.blockDim.x)
    ty = cuda.threadIdx.y
    
    tid = tx
    
    ind_x = tid
    ind_y = bx
    ind_z = by
    
    ind_voxel = (ind_z*IMGSIZy+ind_y)*IMGSIZx+ind_x
    
    total_sum         = 0.0
    total_sensitivity = 0.0
    
    for a in range(angleStart, angleEnd):
        u_term    = 0.0
        
        s         = d_index[a]
        theta     = d_angles[s]
        sin_theta = math.sin(theta)
        cos_theta = math.cos(theta)
        
        #(x0,y0,z0) - source position
        x0 = sourceR*sin_theta
        z0 = sourceR*cos_theta
        y0 = sourceY
        
        #(x1,y1,z1) - center of voxel
        x1 = (ind_x+0.5)*Vsize_x + x_p0
        y1 = (ind_y+0.5)*Vsize_y + y_p0
        z1 = (ind_z+0.5)*Vsize_z + z_p0
        
        #Check FDK paper for this weight factor. This weight can be set to 1, in a simple case
        depth_weight = (x0*x0+y0*y0+z0*z0)/((x0-x1)*(x0-x1) + (y0-y1)*(y0-y1)+(z0-z1)*(z0-z1))
        
        #Do NOT Rotate (x0,y0,z0)  -theta  around the y-axis
        y0r =y0
        x0r =x0
        z0r =z0
        
        #Do NOT Rotate (x1,y1,z1)  -theta around the y-axis
        y1r = y1
        z1r = z1
        x1r = x1
        
        if (z1r != z0r):
            t = (-detectorR - z0r) / (z1r - z0r)
            x2 = x0r + (x1r - x0r) * t
            y2 = y0r + (y1r - y0r) * t
            
            weight = 1.0
            
            # BACKPROJECTION USING INTERPOLATION
            # Calculate the continuous position (in bin_index coordinate) of the projection of voxel in the detector plane.
            imb = ((float)(x2 - x_d0)/Bsize_x)
            jmb = ((float)(y2 - y_d0)/Bsize_y)
            
            ilb = (float)(math.floor(imb))
            if (imb < (ilb+0.5)):
                ilb = ilb - 1
            
            jlb = (float)(math.floor(jmb))
            if ( jmb < (jlb+0.5)):
                jlb = jlb - 1
            
            fracI = imb - (ilb+0.5)
            fracJ = jmb - (jlb+0.5)
            
            d1 = 0
            d2 = 0
            d1_sen = 0
            d2_sen = 0
        
            # Interpolation
            if ((ilb < BINSx) and (ilb >= 0) and (jlb < BINSy) and (jlb >= 0)):
                bin_ind = ilb*BINSy + jlb
                d1      = (1-fracI) * d_prjbuf[int((a-angleStart)*BINSx*BINSy + bin_ind)]
                d1_sen  = (1-fracI) 

            if ((ilb < BINSx-1) and (ilb >= -1) and (jlb < BINSy) and (jlb >= 0)):
                bin_ind = (ilb + 1)* BINSy+ jlb
                d1      = d1 + fracI * d_prjbuf[int((a-angleStart)*BINSx*BINSy + bin_ind)]
                d1_sen  = d1_sen + fracI 

            if ((ilb < BINSx) and (ilb >= 0) and (jlb < BINSy-1) and (jlb >= -1)):
                bin_ind = ilb* BINSy + jlb + 1
                d2      = (1-fracI) * d_prjbuf[int((a-angleStart)*BINSx*BINSy + bin_ind)]
                d2_sen   =  1-fracI 

            if ((ilb<BINSx-1) and (ilb>=-1) and (jlb<BINSy-1) and (jlb>=-1)):
                bin_ind = (ilb + 1) * BINSy +  jlb + 1
                d2 = d2 + fracI  * d_prjbuf[int((a-angleStart)*BINSx*BINSy + bin_ind)]
                d2_sen = d2_sen + fracI
            
            u_term    = (1 - fracJ) * d1 + fracJ * d2
            u_term    = u_term*Vsize_z*depth_weight
            
            u_sensitivity = ((1-fracJ)*d1_sen+fracJ*d2_sen)
            u_sensitivity = u_sensitivity *Vsize_z*depth_weight
            
            total_sum         = total_sum + (u_term*weight)
            total_sensitivity = total_sensitivity+(u_sensitivity*weight)
    
    u_term    = 0
    beta_term = 0
    
    if(total_sensitivity != 0):
        u_term    = (total_sum/total_sensitivity)
        beta_term = (beta*d_prior[ind_voxel])/total_sensitivity
    
    d_objbuf[ind_voxel] = d_objbuf[ind_voxel]+lambda_parameter*(u_term+beta_term)
    if(d_objbuf[ind_voxel] < 0):
        d_objbuf[ind_voxel] = 0
    #if(d_objbuf[ind_voxel] > 0.1):
    #    d_objbuf[ind_voxel] = 0
    
    return

@cuda.jit
def backprj_gpu_manyviews_SBP(d_objbuf, d_prjbuf, d_index, d_angles, angleStart, angleEnd , nbatchIDx):
    # Block id in a 1D grid
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y
    
    tx = cuda.threadIdx.x + (nbatchIDx*cuda.blockDim.x)
    ty = cuda.threadIdx.y
    
    tid = tx
    
    ind_x = tid
    ind_y = bx
    ind_z = by
    
    ind_voxel = (ind_z*IMGSIZy+ind_y)*IMGSIZx+ind_x
    
    total_sum = 0.0
    
    for a in range(angleStart, angleEnd):
        u_term    = 0.0
        
        s         = d_index[a]
        theta     = d_angles[s]
        sin_theta = math.sin(theta)
        cos_theta = math.cos(theta)
        
        #(x0,y0,z0) - source position
        x0 = sourceR*sin_theta
        z0 = sourceR*cos_theta
        y0 = sourceY
        
        #(x1,y1,z1) - center of voxel
        x1 = (ind_x+0.5)*Vsize_x + x_p0
        y1 = (ind_y+0.5)*Vsize_y + y_p0
        z1 = (ind_z+0.5)*Vsize_z + z_p0
        
        #Check FDK paper for this weight factor. This weight can be set to 1, in a simple case
        depth_weight = (x0*x0 + y0*y0 + z0*z0)/((x0-x1)*(x0-x1) + (y0-y1)*(y0-y1)+(z0-z1)*(z0-z1))
        
        y0r = y0
        x0r = x0
        z0r = z0
        
        y1r = y1
        z1r = z1
        x1r = x1
        
        if (z1r != z0r):
            t = (-detectorR - z0r) / (z1r - z0r)
            x2 = x0r + (x1r - x0r) * t
            y2 = y0r + (y1r - y0r) * t
            
            weight = 1.0
            
            # BACKPROJECTION USING INTERPOLATION
            # Calculate the continuous position (in bin_index coordinate) of the projection of voxel in the detector plane.
            imb = ((float)(x2 - x_d0)/Bsize_x)
            jmb = ((float)(y2 - y_d0)/Bsize_y)
            
            ilb = (float)(math.floor(imb))
            if (imb < (ilb+0.5)):
                ilb = ilb - 1
            
            jlb = (float)(math.floor(jmb))
            if ( jmb < (jlb+0.5)):
                jlb = jlb - 1

            fracI = imb - (ilb+0.5)
            fracJ = jmb - (jlb+0.5)
            
            d1 = 0
            d2 = 0
            
            # Interpolation
            if ((ilb < BINSx) and (ilb >= 0) and (jlb < BINSy) and (jlb >= 0)):
                bin_ind = ilb*BINSy + jlb
                d1      = (1-fracI) * d_prjbuf[int((a-angleStart)*BINSx*BINSy + bin_ind)]
            
            if ((ilb < BINSx-1) and (ilb >= -1) and (jlb < BINSy) and (jlb >= 0)):
                bin_ind = (ilb + 1)* BINSy+ jlb
                d1      = d1 + fracI * d_prjbuf[int((a-angleStart)*BINSx*BINSy + bin_ind)]
            
            if ((ilb < BINSx) and (ilb >= 0) and (jlb < BINSy-1) and (jlb >= -1)):
                bin_ind = ilb* BINSy + jlb + 1
                d2      = (1-fracI) * d_prjbuf[int((a-angleStart)*BINSx*BINSy + bin_ind)]
            
            if ((ilb<BINSx-1) and (ilb>=-1) and (jlb<BINSy-1) and (jlb>=-1)):
                bin_ind = (ilb + 1) * BINSy +  jlb + 1
                d2 = d2 + fracI  * d_prjbuf[int((a-angleStart)*BINSx*BINSy + bin_ind)]
            
            u_term    = (1 - fracJ) * d1 + fracJ * d2
            u_term    = u_term*Vsize_z*depth_weight
            total_sum = total_sum + (u_term*weight)
        
    d_objbuf[ind_voxel] = d_objbuf[ind_voxel]+total_sum        
    return


def bprojectCB_GPU_SBP(d_objbuf, d_prjbuf, d_index, d_angles, angleStart, angleEnd):
    BACKPRJ_THREAD = BACKPRJ_ThreX, BACKPRJ_ThreY
    BACKPRJ_GRID   = BACKPRJ_GridX, BACKPRJ_GridY
    
    for nbatchIDx in range(nBatchXdim):
        backprj_gpu_manyviews_SBP[BACKPRJ_GRID, BACKPRJ_THREAD](d_objbuf, d_prjbuf, d_index, d_angles, angleStart, angleEnd , nbatchIDx)
        cuda.synchronize()
    return

@njit(parallel=True)
def temp_fun1(angleStart, b_size, sub_b_size, host_prj_allangle):
    host_prj_sub = np.zeros(sub_b_size)
    for i in prange(sub_b_size):
        host_prj_sub[i]   = host_prj_allangle[angleStart*b_size+i]
    return host_prj_sub

@njit(parallel=True)
def temp_fun2(subset_num, f_size, host_capL):
    temp = np.zeros(f_size)
    for i in prange(f_size):
        temp[i] = subset_num*host_capL[i]
    return temp

@cuda.jit
def G_Fessler_prior(RDD, RD, estbuf, delta, z_xy_ratio, nbatchIDx):
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y
    
    # Thread index
    tx = cuda.threadIdx.x + (nbatchIDx*cuda.blockDim.x)
    ty = cuda.threadIdx.y
    
    cent = 1
    tid  = tx
    
    # Calculate the index of the voxel being considered
    # ind_x = nbatchIDx*((int)(IMGSIZx/h_nBatchXdim))+ tid
    ind_x = tid
    ind_y = bx
    ind_z = by
    
    ind_voxel=int((ind_z*IMGSIZy+ind_y)*IMGSIZx+ind_x)  #(if prj is scanner data, need x_y_flip)
    #ind_voxel=(ind_z*IMGSIZx+ind_x)*IMGSIZy+ind_y;
    
    for ind_nr_z  in range(ind_z-1, ind_z+2):
        for ind_nr_y in range(ind_y-1, ind_y+2):
            for ind_nr_x in range(ind_x-1, ind_x+2):
                distance = math.sqrt(float((ind_nr_x-ind_x)*(ind_nr_x-ind_x)+(ind_nr_y-ind_y)*(ind_nr_y-ind_y)+(ind_nr_z-ind_z)*(ind_nr_z-ind_z)*z_xy_ratio*z_xy_ratio))
                
                if (distance == 0.0):
                    distance = 1.0
                    cent     = 0
                
                if ( ind_nr_x<0  or ind_nr_y<0 or ind_nr_z<0 or ind_nr_x>(IMGSIZx-1) or ind_nr_y>(IMGSIZy-1) or ind_nr_z>(IMGSIZz-1) ):
                    ind_nr = int(ind_voxel)
                else:
                    ind_nr = int(ind_nr_x + ind_nr_y*IMGSIZx + ind_nr_z*IMGSIZx*IMGSIZy)
                
                diff        = estbuf[ind_voxel]-estbuf[ind_nr]
                denominator = 1.0+abs(diff/delta)
                RDD_tmp     = cent*(1.0/distance)/denominator
                
                RDD[ind_voxel] = RDD[ind_voxel] + RDD_tmp
                RD[ind_voxel]  = RD[ind_voxel]  + RDD_tmp*diff
                
                cent = 1 # reset cent
    return

@cuda.jit
def G_Huber_prior_sart(priorbuf, estbuf, delta, nbatchIDx):
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y
    
    # Thread index
    tx = cuda.threadIdx.x + (nbatchIDx*cuda.blockDim.x)
    ty = cuda.threadIdx.y
    
    cent = 1
    tid  = tx
    
    # Calculate the index of the voxel being considered
    # ind_x = nbatchIDx*((int)(IMGSIZx/h_nBatchXdim))+ tid
    ind_x = tid
    ind_y = bx
    ind_z = by
    
    ind_voxel = int((ind_z*IMGSIZy+ind_y)*IMGSIZx+ind_x)  #(if prj is scanner data, need x_y_flip)
    #ind_voxel=(ind_z*IMGSIZx+ind_x)*IMGSIZy+ind_y;
    
    for ind_nr_z  in range(ind_z-1, ind_z+2):
        for ind_nr_y in range(ind_y-1, ind_y+2):
            for ind_nr_x in range(ind_x-1, ind_x+2):
                distance = math.sqrt(float((ind_nr_x-ind_x)*(ind_nr_x-ind_x)+(ind_nr_y-ind_y)*(ind_nr_y-ind_y)+(ind_nr_z-ind_z)*(ind_nr_z-ind_z)))
                
                if (distance == 0.0):
                    distance = 1.0
                
                if ( ind_nr_x<0  or ind_nr_y<0 or ind_nr_z<0 or ind_nr_x>(IMGSIZx-1) or ind_nr_y>(IMGSIZy-1) or ind_nr_z>(IMGSIZz-1) ):
                    ind_nr = int(ind_voxel)
                else:
                    ind_nr = int(ind_nr_x + ind_nr_y*IMGSIZx + ind_nr_z*IMGSIZx*IMGSIZy)
                
                diff        = estbuf[ind_voxel]-estbuf[ind_nr]
                denominator = math.sqrt(1.0+(diff/delta)*(diff/delta))
                
                priorbuf[ind_voxel] = priorbuf[ind_voxel] + (1.0/distance)*diff/denominator
    return    

def prior_GPU_SART(d_prior, d_est, delta):
    BACKPRJ_THREAD = BACKPRJ_ThreX, BACKPRJ_ThreY
    BACKPRJ_GRID   = BACKPRJ_GridX, BACKPRJ_GridY
    
    if (delta == 0):
        print("delta cannot be ZERO !!")
        exit(1)
    
    for nbatchIDx in range(0, nBatchXdim):
        G_Huber_prior_sart[BACKPRJ_GRID, BACKPRJ_THREAD](d_prior, d_est, delta, nbatchIDx)
        # Check out the content of this kernel in file ConebeamCT_kernel.cu
        cuda.synchronize()
    return

def prior_GPU_OSTR(d_RDD, d_RD, d_est, delta, z_xy_ratio):
    BACKPRJ_THREAD = BACKPRJ_ThreX, BACKPRJ_ThreY
    BACKPRJ_GRID   = BACKPRJ_GridX, BACKPRJ_GridY
    
    if (delta == 0):
        print("delta cannot be ZERO !!")
        exit(1)
    
    for nbatchIDx in range(0, nBatchXdim):
        G_Fessler_prior[BACKPRJ_GRID, BACKPRJ_THREAD](d_RDD, d_RD, d_est, delta, z_xy_ratio, nbatchIDx)
        # Check out the content of this kernel in file ConebeamCT_kernel.cu
        cuda.synchronize()
    return



In [6]:
# File Reading Code

def load_prj_ima():
    b_size = BINSx*BINSy
    flag2  = 0
    prj_allangle  = np.zeros(BINSx*BINSy*ANGLES)
    
    print(BINSx*BINSy, ANGLES, prj_allangle.shape)
    x = []
    y = []

    proj_paths = glob.glob("/media/pranjal/de24af8d-2361-4ea2-a07a-1801b54488d9/DBT_recon_data/"+projection_name+"/CE*.IMA")
    for p in proj_paths:
        if '.0000.' in p:
            continue
        a    = pydicom.dcmread(p)
        temp = a.pixel_array.T
        temp = np.log(10000)-np.log(temp)
        
        # Sharpening filter
        temp = unsharp_mask(temp, radius=5, amount=1, preserve_range=True)
        
        val = filters.threshold_otsu(temp)
        temp[temp < val] = 0
        
        temp = np.fliplr(temp)
        temp = temp[-BINSy:]
        
        temp = temp.flatten()
        temp = x_y_flip(temp)
        
        x.append(temp)
        y.append(float(a[0x00181530].value))
    y = np.array(y)*np.pi/180
    
    print("length x ", len(x), " ", len(y))
    y, x = zip(*sorted(zip(y, x)))
    for j in range(len(x)):
        print("Proj ", j)
        flag2 = j
        for i in range(0, BINSx*BINSy):
            prj_allangle[flag2*BINSx*BINSy + i]  = x[j][i]
    
    return prj_allangle, y

def import_param():
    for i in range(ANGLES):
        index[i] = i
    return index

def load_prj():
    b_size = BINSx*BINSy
    flag2  = 0
    
    prj_allangle  = np.zeros(BINSx*BINSy*ANGLES)
    scat_allangle = np.zeros(BINSx*BINSy*ANGLES)
    
    for viewangle in range(ANGLES):
        s        = viewangle + 1
        filename = basepath + filepath+str(s).zfill(4)#+'.raw'
        
        with open(filename, 'rb') as f:
            primary_plus_scatter  = np.fromfile(f, dtype=np.float32)
            host_prj_temp1        = primary_plus_scatter[:b_size]
            host_prj_temp2        = primary_plus_scatter[b_size:]
        
        host_prj_1view_temp = x_y_flip(host_prj_temp1)
        host_sct_1view_temp = x_y_flip(host_prj_temp2)
        
        print(host_prj_1view_temp.shape)
        
        # all angle together
        for i in range(0, BINSx*BINSy):
            prj_allangle[flag2*BINSx*BINSy + i]  = host_prj_1view_temp[i]
            scat_allangle[flag2*BINSx*BINSy + i] = host_sct_1view_temp[i]
        
        flag2 = flag2+1
    return prj_allangle, scat_allangle
    
def load_prj_std(data_type):
    b_size = BINSx*BINSy
    flag2  = 0
    
    prj_allangle  = np.zeros(BINSx*BINSy*ANGLES)
    #scat_allangle = np.zeros(BINSx*BINSy*ANGLES)
    
    for viewangle in range(ANGLES):
        s        = viewangle + 1
        
        if data_type   == 0:
            #filename = basepath + 'OSTR_prelog/'+projection_name+str(s).zfill(4)#+".raw"
            filename = basepath + 'Projections_Renamed_Seg/'+projection_name+str(s).zfill(4)#+".raw"
        elif data_type == 1:
            filename = basepath + 'OSTR_scatter/'+scatter_name+str(s).zfill(4)#+".raw"
        else:
            filename = basepath + 'OSTR_blank/'+blank_name+str(s).zfill(4)#+".raw"
        
        #print(filename)
        with open(filename, 'rb') as f:
            #data  = np.load(f)
            data  = np.fromfile(f, dtype=np.float32)
            # If doign SART
            #data  = np.log(10000) - np.log(data)
            
            #print(data.shape)
            #data  = np.reshape(data, (1400, 3584))
            #data  = data[:1400, :]
            #data  = np.flip(data, 0)
            #data  = data.flatten()
            #print(data.shape)
                #data  = np.fromfile(f, dtype=np.float32)
        #np.save(filename+'.npy', data)
        proj = x_y_flip(data)
        
        # all angle together
        for i in range(0, BINSx*BINSy):
            prj_allangle[flag2*BINSx*BINSy + i]  = proj[i]
        
        flag2 = flag2+1
    
    return prj_allangle


In [7]:
# Settings Part


projection_name = "CE27"#"CE26.3584x1000."#"OSTR_LE.3584x1400."
scatter_name    = "OSTR_scatter.3584x1400." 
blank_name      = "OSTR_blank.3584x1400."

IMGSIZx = 2600
IMGSIZy = 1200
IMGSIZz = 48
f_size  = IMGSIZx*IMGSIZy*IMGSIZz

BINSx   = 3584
#BINSy   = 2816
BINSy   = 1600

Vsize_x = 0.085
Vsize_y = 0.085
Vsize_z = 1

x_p0    = -106.25
y_p0    = -107.1
z_p0    = -30.0

x_d0    = -152.32
y_d0    = -137.7

Bsize_x = 0.085
Bsize_y = 0.085
b_size  = BINSx*BINSy

ANGLES  = 25
index   = []
angles  = []


detectorR = 47.0
sourceR   = 608.5
sourceY   = 0.0

# Tuning Parameters
beta  = 1000
delta = 0.03
#####################

iter_num   = 5
subset_num = 5

IO_Iter = 0
method  = 0


BACKPRJ_ThreX = 390
BACKPRJ_ThreY = 1
BACKPRJ_GridX = 1000
BACKPRJ_GridY = 48
nBatchXdim    = 8

nBatchBINSx = 1
nBatchBINSy = 1

PRJ_ThreX = 16
PRJ_ThreY = 16
PRJ_GridX = 224
PRJ_GridY = 100


f_size     = IMGSIZx*IMGSIZy*IMGSIZz
all_b_size = ANGLES*BINSx*BINSy
sub_b_size = BINSx*BINSy

In [ ]:
# Code for doing SART Recon
#h_angles
host_prj_allangle_backup, h_angles   = load_prj_ima()
h_angles = list(h_angles)

h_index   = np.array(list(range(0, 25)))

d_angles = cuda.to_device(h_angles)
d_index  = cuda.to_device(h_index)

angleStart = 0
angleEnd   = 25


host_prj_allangle   = copy.deepcopy(host_prj_allangle_backup) #load_prj_std(0) # Load prelog  data

host_prj_allangle   = regroup_prj(host_prj_allangle)

index_tmp = np.zeros(ANGLES)
for i in range(0, ANGLES):
    index_tmp[i] = h_index[i]

flag           = 0
ANGLES_per_sub = int(ANGLES/subset_num)
h_index        = np.zeros(ANGLES, dtype=int)

for i in range(0, subset_num):
    for j in range(0, ANGLES_per_sub):
        h_index[flag] = index_tmp[j*subset_num+i]
        flag          = flag + 1
        


sub_b_size     = ANGLES_per_sub*b_size

print('sub_b_size is ', sub_b_size)
print('delta is ',      delta)
print("Indexes are ",   h_index)

In [ ]:
# Recon Loop

beta_array       = [-0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0]
delta_array      = [0.001, 0.0005, 0.0001, 0.00005, 0.00001]
lambda_parameter = 0.9

for beta in beta_array:
    for delta in delta_array:
        print("Reconstructing ", beta, delta)
        
        d_line_sub     = cuda.device_array(int(sub_b_size))
        d_normprj      = cuda.device_array(int(sub_b_size))

        d_index        = cuda.to_device(h_index)

        host_est      = np.zeros(f_size, np.float32)
        d_est         = cuda.to_device(host_est)

        host_prj_est  = np.zeros(f_size, np.float32)
        prj_est       = cuda.to_device(host_prj_est)

        host_prj_est  = np.zeros(f_size, np.float32)
        prj_est       = cuda.to_device(host_prj_est)

        d_prior       = np.zeros(f_size, np.float32)
        d_prior       = cuda.to_device(d_prior)

        d_diff_line_sub = cuda.device_array(int(sub_b_size))
        d_normprj_sub   = cuda.device_array(int(sub_b_size))
        d_prj_est_sub   = cuda.device_array(int(sub_b_size))
        #d_prj_buf_sub   = cuda.device_array(int(sub_b_size))

        for i in range(0, 5):
            #print("Iteration ", i)
            for a in range(0, 5):
                angleStart = a*ANGLES_per_sub
                angleEnd   = (a+1)*ANGLES_per_sub

                host_prj_sub  = temp_fun1(angleStart, b_size, sub_b_size, host_prj_allangle)
                d_prj_buf_sub = cuda.to_device(host_prj_sub)

                d_prior       = np.zeros(f_size, np.float32)
                d_prior       = cuda.to_device(d_prior)

                prior_GPU_SART(d_prior, d_est, delta)

                fprojectCB_1R_GPU_SART_cos(
                    d_est,
                    d_prj_est_sub,
                    d_normprj_sub,
                    d_angles,
                    d_index,
                    angleStart,
                    angleEnd)

                #d_normprj_sub
                h_diff_line_sub  = d_diff_line_sub.copy_to_host()
                h_normprj_sub    = d_normprj_sub.copy_to_host()
                h_prj_est_sub    = d_prj_est_sub.copy_to_host()

                #h_normprj_sub[h_normprj_sub  < 0.5] = 10000

                result_diff = SART_prj_diff(h_diff_line_sub,
                    host_prj_sub,
                    h_prj_est_sub,
                    h_normprj_sub,
                    h_index,
                    angleStart,
                    angleEnd)

                #result_diff[result_diff > 0.5] = 0
                #result_diff[result_diff > 100] = 0
                #result_diff = np.nan_to_num(np.divide(host_prj_sub - h_prj_est_sub, h_normprj_sub))

                d_diff_line_sub = cuda.to_device(result_diff)

                bprojectCB_4B_GPU_R_SART (d_est, d_diff_line_sub, d_prior,
                                          d_index,
                                          d_angles,
                                          angleStart,
                                          angleEnd,
                                          lambda_parameter, beta)

                d_prj_est_sub = np.zeros(d_prj_est_sub.shape)
                d_prj_est_sub = cuda.to_device(d_prj_est_sub)

                d_normprj_sub = np.zeros(d_normprj_sub.shape)
                d_normprj_sub = cuda.to_device(d_normprj_sub)

                d_prj_buf_sub = np.zeros(d_prj_buf_sub.shape)
                d_prj_buf_sub = cuda.to_device(d_prj_buf_sub)

        host_est = d_est.copy_to_host()
        host_est.astype('float32').tofile('/media/pranjal/BackupPlus/DBT-HUBER-VOL-DEBLUR/'+projection_name+'_'+str(IMGSIZx)+'x'+str(IMGSIZy)+'x'+str(IMGSIZz)+'.'+str(i)+'_'+str(delta)+'_'+str(beta)+'.raw')

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

import matplotlib.pyplot as plt
import numpy as np


device = torch.device("cuda:0")

in1 = np.ones([10, 1, 256, 256])
in2 = np.ones([10, 1])

in1 = torch.tensor(in1, device=device).float()
in2 = torch.tensor(in2, device=device).float()


# Define model
class MyUnet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        filter1 = 32
        filter2 = 64
        filter3 = 128
        filter4 = 256
        filter5 = 512
    
        self.dense_block = nn.Sequential(nn.Linear(1, 128),
                                         nn.LeakyReLU(),
                                         nn.Linear(128, 32),
                                         nn.LeakyReLU(),
                                         nn.Linear(32, 1),
                                         nn.LeakyReLU()
                                        )
        
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(filter1, filter1, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(filter1, filter2, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(filter2, filter2, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.pool2 = nn.MaxPool2d(2, stride=2)
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(filter2, filter3, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(filter3, filter3, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.pool3 = nn.MaxPool2d(2, stride=2)
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(filter3, filter4, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(filter4, filter4, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.pool4 = nn.MaxPool2d(2, stride=2)
        self.conv_block5 = nn.Sequential(
            nn.Conv2d(filter4, filter5, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(filter5, filter5, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.pool5 = nn.MaxPool2d(2, stride=2)
        
        self.upsample1   = nn.Upsample(scale_factor=2, mode='bilinear')
        self.upsample2   = nn.Upsample(scale_factor=2, mode='bilinear')
        self.upsample3   = nn.Upsample(scale_factor=2, mode='bilinear')
        self.upsample4   = nn.Upsample(scale_factor=2, mode='bilinear')
        
        self.conv_block_merge1 = nn.Sequential(
            nn.Conv2d(filter5, filter4, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.conv_block_merge2 = nn.Sequential(
            nn.Conv2d(filter4, filter3, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.conv_block_merge3 = nn.Sequential(
            nn.Conv2d(filter3, filter2, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.conv_block_merge4 = nn.Sequential(
            nn.Conv2d(filter2, filter1, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        
        
        self.conv_block6 = nn.Sequential(
            nn.Conv2d(filter5, filter4, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(filter4, filter4, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.conv_block7 = nn.Sequential(
            nn.Conv2d(filter4, filter3, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(filter3, filter3, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.conv_block8 = nn.Sequential(
            nn.Conv2d(filter3, filter2, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(filter2, filter2, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.conv_block9 = nn.Sequential(
            nn.Conv2d(filter2, filter1, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(filter1, filter1, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        self.conv_block10 = nn.Sequential(
            nn.Conv2d(filter1, 1, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2))
        
        self.d1 = nn.Dropout(0.5)
        self.d2 = nn.Dropout(0.5)
        
    def forward(self, x, y):
        hash_val = self.dense_block(y)
        hash_val = hash_val.view(-1, 1, 1, 1)
        
        x1 = self.conv_block1(x)
        x1 = torch.mul(x1, hash_val)
        p1 = self.pool1(x1)
        
        x2 = self.conv_block2(p1)
        x2 = torch.mul(x2, hash_val)
        p2 = self.pool2(x2)
        
        x3 = self.conv_block3(p2)
        x3 = torch.mul(x3, hash_val)
        p3 = self.pool3(x3)
        p3 = self.d1(p3)
        
        x4 = self.conv_block4(p3)
        x4 = torch.mul(x4, hash_val)
        p4 = self.pool4(x4)
        p4 = self.d2(p4)
        
        x5 = self.conv_block5(p4)
        x5 = torch.mul(x5, hash_val)
        
        
        
        u1 = self.upsample1(x5)
        m6 = self.conv_block_merge1(u1)
        m6 = torch.cat((x4, m6), 1)
        x6 = self.conv_block6(m6)
        x6 = torch.mul(x6, hash_val)
        
        u2 = self.upsample2(x6)
        m7 = self.conv_block_merge2(u2)
        m7 = torch.cat((x3, m7), 1)
        x7 = self.conv_block7(m7)
        x7 = torch.mul(x7, hash_val)
        
        u3 = self.upsample3(x7)
        m8 = self.conv_block_merge3(u3)
        m8 = torch.cat((x2, m8), 1)
        x8 = self.conv_block8(m8)
        x8 = torch.mul(x8, hash_val)
        
        u4 = self.upsample4(x8)
        m9 = self.conv_block_merge4(u4)
        m9 = torch.cat((x1, m9), 1)
        x9 = self.conv_block9(m9)
        x9 = self.conv_block10(x9)
        
        out = torch.sub(x, x9)
        
        return out

class RatingModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU()
        )
        self.pool1 = nn.MaxPool2d(2, stride=2)
                             
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU()
        )
        self.pool2 = nn.MaxPool2d(2, stride=2)
        
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(32, 8, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU()
        )
        self.pool3 = nn.MaxPool2d(2, stride=2)
        
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(8, 1, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU()
        )
        self.pool4 = nn.MaxPool2d(2, stride=2)
        self.out   = nn.Linear(256, 1)
        
        #self.out = Linear()
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.pool1(x)
        x = self.conv_block2(x)
        x = self.pool2(x)
        x = self.conv_block3(x)
        x = self.pool3(x)
        x = self.conv_block4(x)
        x = self.pool4(x)
        x = torch.flatten(x)
        
        x = self.out(x)
        x = torch.sigmoid(x)
        
        return x

# Initialize model
#model = MyUnet()
#model.cuda()

def init_normal(m):
    if type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight)
    if type(m) == nn.Linear:
        nn.init.kaiming_normal_(m.weight)

        

class RatingModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(4),
            nn.MaxPool2d(2, stride=2))
        
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(4),
            nn.MaxPool2d(2, stride=2))
        
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(4),
            nn.MaxPool2d(2, stride=2))
        
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(4),
            nn.MaxPool2d(2, stride=2))
        
        self.conv_block5 = nn.Sequential(
            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(4),
            nn.MaxPool2d(2, stride=2))
        
        self.out = nn.Sequential(nn.Linear(256, 1),
                                 nn.Sigmoid())
        
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)
        
        x = x.view(-1, 256)
        x = self.out(x)
        
        return x

model = RatingModel()
model.cuda()

In [26]:
# Lesion Location information Array hash Creation

h = {}
s = open("/home/pranjal/lesion_array", "r").read()
s = s.split("\n")
s = s[:-1]
count = 0
for t in s:
    #print(t, "p")
    a = t.strip().split()
    #print(a[0], " p ", a[1], a[2], a[3])
    i = int(a[0])
    x = int(a[1].replace(",", ""))
    y = int(a[2].replace(",", ""))
    z = int(a[3].replace(",", ""))
    #print(i, x, y, z)
    h[count] = [i, x, y, z]
    count = count+1
np.save('lesion_array_hash.npy', h)

In [88]:
# For training the Rating CNN


import glob
import random

trainx = np.zeros([160, 1, 256, 256])
trainy = np.zeros([160, 1])
valx   = np.zeros([70, 1, 256, 256])
valy   = np.zeros([70, 1])

train_list = [1, 3, 6, 7, 11, 12, 13, 16, 17, 31, 33, 35, 37, 39, 41, 43, 50, 52, 55,  68, 69, 70, 71, 72, 73, 76]
val_list   = [19, 21, 23, 25, 27, 29, 61, 62, 64, 65]
test_list  = [10, 44, 45, 47, 54, 58, 59, 60, 66, 67, 74, 75]

traincount = 0
valcount   = 0

allfiles = glob.glob("/home/pranjal/newrecon2/newrecon/dbt_real_ratings/x_*.npy")
for f in allfiles:
    x1 = np.load(f)
    y1 = -1*np.load(f.replace("x_", "z_"))
    
    k = int(f.split("/")[-1].split("_")[1][:-4])
    #print(k, h[k])
    
    x1 = np.expand_dims(x1, axis=-1)
    y1 = np.expand_dims(y1, axis=-1)
    
    #print(x1.shape, y1.shape)
    
    if h[k][0] in train_list:
        #print(x1.shape, y1.shape, "Train")
        if x1.shape[0] == 0:
            continue
        
        trainx[traincount:traincount+x1.shape[0], 0, :, :] = x1[:, :, :, 0]
        trainy[traincount:traincount+x1.shape[0], :] = y1
        #trainx.append(x1)
        #trainy.append(y1)
        traincount = traincount+x1.shape[0]
    elif h[k][0] in val_list:
        #print(x1.shape, y1.shape, "Val")
        if x1.shape[0] == 0:
            continue
        
        valx[valcount:valcount+x1.shape[0], 0, :, :] = x1[:, :, :, 0]
        valy[valcount:valcount+x1.shape[0], :] = y1
        #valx.append(x1)
        #valy.append(y1)
        valcount = valcount+x1.shape[0]

print(traincount, valcount)

160 70


In [52]:
# For training the Rating CNN using Real DBT slices Data

#10336 4148
#6920 2770

#2160 864


trainx = np.zeros([153, 256, 256, 1])
trainy = np.zeros([153, 1])
trainv = np.zeros([153, 1])

valx   = np.zeros([63, 256, 256, 1])
valy   = np.zeros([63, 1])
valv   = np.zeros([63, 1])

train_list = [1, 3, 6, 7, 11, 12, 13, 16, 17, 31, 33, 35, 37, 39, 41, 43, 50, 52, 55,  68, 69, 70, 71, 72, 73, 76]
val_list   = [19, 21, 23, 25, 27, 29, 61, 62, 64, 65]
test_list  = [10, 44, 45, 47, 54, 58, 59, 60, 66, 67, 74, 75]

traincount = 0
valcount   = 0

allfiles = glob.glob("/home/pranjal/newrecon2/newrecon/dbt_real_cho_data/noisy_x_*.npy")
for f in allfiles:
    #print(f)
    x1 = np.load(f)
    y1 = -1*np.load(f.replace("noisy_x_", "y_"))
    
    #print(f.split("/")[-1].split("_"))
    k = int(f.split("/")[-1].split("_")[2][:-4])
    #print(k, h[k])
    
    x1 = np.expand_dims(x1, axis=-1)
    #y1 = np.expand_dims(y1, axis=-1)
    
    #print(x1.shape, y1.shape)
    
    if h[k][0] in train_list:
        #print(x1.shape, y1.shape, "Train")
        if x1.shape[0] == 0:
            continue
        
        trainx[traincount:traincount+x1.shape[0], :, :, :] = x1
        trainy[traincount:traincount+x1.shape[0], 0] = 1
        #trainv[traincount:traincount+x1.shape[0], 0] = y1
        
        #trainx.append(x1)
        #trainy.append(y1)
        traincount = traincount+x1.shape[0]
    elif h[k][0] in val_list:
        #print(x1.shape, y1.shape, "Val")
        if x1.shape[0] == 0:
            continue
        
        valx[valcount:valcount+x1.shape[0], :, :, :] = x1
        valy[valcount:valcount+x1.shape[0], 0] = 1
        #valv[valcount:valcount+x1.shape[0], 0] = y1
        
        #valx.append(x1)
        #valy.append(y1)
        valcount = valcount+x1.shape[0]
    #print(traincount, valcount)

print(traincount, valcount)

# trainx_lesion = copy.deepcopy(trainx)
# trainy_lesion = copy.deepcopy(trainy)

share = 10
allfiles = glob.glob("/home/pranjal/newrecon2/newrecon/dbt_real_cho_data/noisy_no_x_*.npy")
for f in allfiles:
    #print(f)
    x1 = np.load(f)
    #y1 = -1*np.load(f.replace("x_", "v_"))
    
    #print(f.split("/")[-1].split("_"))
    k = int(f.split("/")[-1].split("_")[3][:-4])
    #print(k, h[k])
    
    x1 = np.expand_dims(x1, axis=-1)
    #y1 = np.expand_dims(y1, axis=-1)
    
    #print(x1.shape, y1.shape, k)
    
    
    if h[k][0] in train_list:
        #print(x1.shape, y1.shape, "Train")
        if x1.shape[0] == 0:
            continue
        
        trainx[traincount:traincount+int(x1.shape[0]/share), :, :, :] = x1[:int(x1.shape[0]/share)]
        trainy[traincount:traincount+int(x1.shape[0]/share), 0] = 0
        #trainv[traincount:traincount+x1.shape[0], 0] = y1
        
        #trainx.append(x1)
        #trainy.append(y1)
        traincount = traincount+int(x1.shape[0]/share)
        #print(traincount, " traincount")
    elif h[k][0] in val_list:
        #print(x1.shape, y1.shape, "Val")
        if x1.shape[0] == 0:
            continue
        
        valx[valcount:valcount+int(x1.shape[0]/share), :, :, :] = x1[:int(x1.shape[0]/share)]
        valy[valcount:valcount+int(x1.shape[0]/share), 0] = 0
        #valv[valcount:valcount+x1.shape[0], 0] = y1
        #valx.append(x1)
        #valy.append(y1)
        valcount = valcount+int(x1.shape[0]/share)
    #print(traincount, valcount)
#     elif h[k][0] in test_list:
#         print("Found in test ", k)
#     else:
#         print("Not found ")


print(traincount, valcount)

68 28
153 63


In [ ]:
from torchsummary import summary
summary(model, (1, 256, 256))

In [90]:
device    = torch.device("cuda:0")

trainx = torch.tensor(trainx, device=device).float()
trainy = torch.tensor(trainy, device=device).float()

valx = torch.tensor(valx, device=device).float()
valy = torch.tensor(valy, device=device).float()

#z = torch.tensor(x, device=device).float()

In [ ]:
from torchsample.modules import ModuleTrainer

model1 = ModuleTrainer(model)

#model = Network()
model1.set_loss(F.mse_loss)
model1.set_optimizer("adam", lr=1.0)


# fit model
model1.fit(trainx, trainy,
          val_data=(valx, valy), num_epoch=5,
          batch_size=32,
          verbose=1)

# evaluate on test data
val_loss = model1.evaluate(valx, valy)

# # predict on input data
# y_pred = model.predict(x_test)


In [7]:
import numpy as np

trainids = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/TRAIN.npy')
valids   = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/VALIDATION.npy')

In [14]:
# All other functions

def dice_coefficient(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coefficient_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)


def dice_coef(y_true, y_pred, smooth, thresh):
    #y_pred =K.cast((K.greater(y_pred,thresh)), dtype='float32')#转换为float型
    #y_pred = y_pred[y_pred > thresh]=1.0
    y_true_f =y_true# K.flatten(y_true)
    y_pred_f =y_pred# K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f,axis=(0,1,2))
    denom =K.sum(y_true_f,axis=(0,1,2)) + K.sum(y_pred_f,axis=(0,1,2))
    return K.mean((2. * intersection + smooth) /(denom + smooth))

def dice(y_true, y_pred):
        return 1-dice_coef(y_true, y_pred, 0.1, 0.5)

def dice_loss(smooth, thresh):
    def dice(y_true, y_pred):
        return 1-dice_coef(y_true, y_pred, smooth, thresh)
    return dice


In [ ]:
import keras

model    = load_model('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/unet-cnn-4-class.h5')
model.compile(optimizer = Adam(lr = 1e-3), loss = dice_coefficient_loss, 
              metrics = ['accuracy', dice_coefficient])

print(model.summary())

In [ ]:
# For training U-Net

import keras

model    = load_model('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/unet-cnn-4-class-dice.h5', custom_objects={'dice_coefficient_loss': dice_loss(smooth=1e-5,thresh=0.5), 'dice_coefficient':dice_coefficient})
model.compile(optimizer = Adam(lr = 1e-5), loss = dice_loss(smooth=1e-5,thresh=0.5), metrics = ['accuracy'])
model.load_weights('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/unet-cnn-4-class-entropy.h5')

#model    = load_model('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/unet-cnn-4-class-dice.h5')

#model.compile(optimizer = Adam(lr = 1e-4), loss = dice_loss(smooth=1e-5,thresh=0.5), metrics = ['accuracy'])
# K.get_session().close()
# K.set_session(tf.Session())
# K.get_session().run(tf.global_variables_initializer())


trainids = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/TRAIN.npy')
valids   = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/VALIDATION.npy')

for i in range(10000):
    np.random.shuffle(trainids)
    for t in trainids[:5]:
        #print(t)
        x            = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-NII/'+t+'.nii.gz'))+1024.0
        x[x > 2000]  =  2000
        x[x < -2000] = -2000
        x = x/1024.0
        
        y = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-SEG-GROUND/'+t+'.mhd')).astype('uint8')
        
        y[y == 3] = 2
        y[y == 4] = 1
        y[y == 5] = 3
        
        x = np.expand_dims(x, axis=-1)
        y = np.expand_dims(y, axis=-1)
        
        y = keras.utils.to_categorical(y, num_classes=4, dtype='uint8')
        
        print(x.shape, y.shape)
        model.fit(x, y, batch_size=4, verbose=True)
    
    if i % 5 == 0:
        print('Evaluating the Model')
        np.random.shuffle(valids)
        for t in valids[:5]:
            #print(t)
            x = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-NII/'+t+'.nii.gz'))+1024.0
            x[x > 2000]  = 2000
            x[x < -2000] = -2000
            x = x/1024.0
            y = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-SEG-GROUND/'+t+'.mhd')).astype('uint8')

            y[y == 3] = 2
            y[y == 4] = 1
            y[y == 5] = 3

            x = np.expand_dims(x, axis=-1)
            y = np.expand_dims(y, axis=-1)

            y = keras.utils.to_categorical(y, num_classes=4, dtype='uint8')

            #print(x.shape, y.shape)        
            print(t, model.evaluate(x, y, batch_size=4))
    
    model.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/unet-cnn-4-class-dice1.h5')

In [ ]:
model    = load_model('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/unet-cnn-4-class-dice.h5', compile=False)
result = model.predict(x, batch_size=4)
print(result.shape)

In [ ]:
# All Imports

from __future__ import print_function, division
import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, merge
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import glob


import numba
from numba import njit, prange

from keras import backend as K
from keras.layers import Layer

import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, Callback, TensorBoard
from keras import backend as keras

from skimage.measure import label
from scipy.io import loadmat
from scipy.ndimage import zoom
#from scipy.misc import imresize
import pywt

import csv
import random
import time
%matplotlib inline  

from scipy import ndimage, misc

import pywt
#import hdf5storage

import scipy.io as sio
from skimage.filters import threshold_otsu

#import pylidc as pl
#from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

import pywt
import numpy as np
#import pydicom
import matplotlib.pyplot as plt
import SimpleITK as sitk
import skimage.io as io
#from sklearn.decomposition import PCA
import collections, numpy
import warnings
from scipy import ndimage, misc
warnings.filterwarnings('ignore')

#import pymrt as mrt
#import pymrt.geometry
import ipyvolume as ipv
import copy


import os
import glob
import uuid
import numpy as np
import LungNet as LN
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from ipdb import set_trace as bp



#from image_gen import ImageDataGenerator
#from load_data import loadDataMontgomery, loadDataJSRT
#from build_model import build_UNet2D_4L

import pandas as pd
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint



import numpy
import warnings
from keras.layers import Convolution3D, Input, merge, RepeatVector, Activation
from keras.models import Model
from keras.layers.advanced_activations import PReLU
from keras import activations, initializers, regularizers
from keras.engine import Layer, InputSpec
from keras.utils.conv_utils import conv_output_length
#from keras.utils.np_utils import conv_output_length
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from keras.engine.topology import Layer
import functools
import tensorflow as tf
import pickle
import time


In [ ]:
# [BLOCK 9] For training the SegNet CNN Model

from keras.models import Model
from keras.layers import Input
from keras.layers.core import Activation, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization

#from layers import MaxPoolingWithArgmax2D, MaxUnpooling2D


def segnet(
        input_shape,
        n_labels,
        kernel=3,
        pool_size=(2, 2),
        output_mode="softmax"):
    # encoder
    inputs = Input(shape=input_shape)

    conv_1 = Convolution2D(64, (kernel, kernel), padding="same")(inputs)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation("relu")(conv_1)
    conv_2 = Convolution2D(64, (kernel, kernel), padding="same")(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    conv_2 = Activation("relu")(conv_2)

    pool_1, mask_1 = MaxPoolingWithArgmax2D(pool_size)(conv_2)

    conv_3 = Convolution2D(128, (kernel, kernel), padding="same")(pool_1)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = Activation("relu")(conv_3)
    conv_4 = Convolution2D(128, (kernel, kernel), padding="same")(conv_3)
    conv_4 = BatchNormalization()(conv_4)
    conv_4 = Activation("relu")(conv_4)

    pool_2, mask_2 = MaxPoolingWithArgmax2D(pool_size)(conv_4)

    conv_5 = Convolution2D(256, (kernel, kernel), padding="same")(pool_2)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = Activation("relu")(conv_5)
    conv_6 = Convolution2D(256, (kernel, kernel), padding="same")(conv_5)
    conv_6 = BatchNormalization()(conv_6)
    conv_6 = Activation("relu")(conv_6)
    conv_7 = Convolution2D(256, (kernel, kernel), padding="same")(conv_6)
    conv_7 = BatchNormalization()(conv_7)
    conv_7 = Activation("relu")(conv_7)

    pool_3, mask_3 = MaxPoolingWithArgmax2D(pool_size)(conv_7)

    conv_8 = Convolution2D(512, (kernel, kernel), padding="same")(pool_3)
    conv_8 = BatchNormalization()(conv_8)
    conv_8 = Activation("relu")(conv_8)
    conv_9 = Convolution2D(512, (kernel, kernel), padding="same")(conv_8)
    conv_9 = BatchNormalization()(conv_9)
    conv_9 = Activation("relu")(conv_9)
    conv_10 = Convolution2D(512, (kernel, kernel), padding="same")(conv_9)
    conv_10 = BatchNormalization()(conv_10)
    conv_10 = Activation("relu")(conv_10)

    pool_4, mask_4 = MaxPoolingWithArgmax2D(pool_size)(conv_10)

    conv_11 = Convolution2D(512, (kernel, kernel), padding="same")(pool_4)
    conv_11 = BatchNormalization()(conv_11)
    conv_11 = Activation("relu")(conv_11)
    conv_12 = Convolution2D(512, (kernel, kernel), padding="same")(conv_11)
    conv_12 = BatchNormalization()(conv_12)
    conv_12 = Activation("relu")(conv_12)
    conv_13 = Convolution2D(512, (kernel, kernel), padding="same")(conv_12)
    conv_13 = BatchNormalization()(conv_13)
    conv_13 = Activation("relu")(conv_13)

    pool_5, mask_5 = MaxPoolingWithArgmax2D(pool_size)(conv_13)
    print("Build enceder done..")

    # decoder

    unpool_1 = MaxUnpooling2D(pool_size)([pool_5, mask_5])

    conv_14 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_1)
    conv_14 = BatchNormalization()(conv_14)
    conv_14 = Activation("relu")(conv_14)
    conv_15 = Convolution2D(512, (kernel, kernel), padding="same")(conv_14)
    conv_15 = BatchNormalization()(conv_15)
    conv_15 = Activation("relu")(conv_15)
    conv_16 = Convolution2D(512, (kernel, kernel), padding="same")(conv_15)
    conv_16 = BatchNormalization()(conv_16)
    conv_16 = Activation("relu")(conv_16)

    unpool_2 = MaxUnpooling2D(pool_size)([conv_16, mask_4])

    conv_17 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_2)
    conv_17 = BatchNormalization()(conv_17)
    conv_17 = Activation("relu")(conv_17)
    conv_18 = Convolution2D(512, (kernel, kernel), padding="same")(conv_17)
    conv_18 = BatchNormalization()(conv_18)
    conv_18 = Activation("relu")(conv_18)
    conv_19 = Convolution2D(256, (kernel, kernel), padding="same")(conv_18)
    conv_19 = BatchNormalization()(conv_19)
    conv_19 = Activation("relu")(conv_19)

    unpool_3 = MaxUnpooling2D(pool_size)([conv_19, mask_3])

    conv_20 = Convolution2D(256, (kernel, kernel), padding="same")(unpool_3)
    conv_20 = BatchNormalization()(conv_20)
    conv_20 = Activation("relu")(conv_20)
    conv_21 = Convolution2D(256, (kernel, kernel), padding="same")(conv_20)
    conv_21 = BatchNormalization()(conv_21)
    conv_21 = Activation("relu")(conv_21)
    conv_22 = Convolution2D(128, (kernel, kernel), padding="same")(conv_21)
    conv_22 = BatchNormalization()(conv_22)
    conv_22 = Activation("relu")(conv_22)

    unpool_4 = MaxUnpooling2D(pool_size)([conv_22, mask_2])

    conv_23 = Convolution2D(128, (kernel, kernel), padding="same")(unpool_4)
    conv_23 = BatchNormalization()(conv_23)
    conv_23 = Activation("relu")(conv_23)
    conv_24 = Convolution2D(64, (kernel, kernel), padding="same")(conv_23)
    conv_24 = BatchNormalization()(conv_24)
    conv_24 = Activation("relu")(conv_24)

    unpool_5 = MaxUnpooling2D(pool_size)([conv_24, mask_1])

    conv_25 = Convolution2D(64, (kernel, kernel), padding="same")(unpool_5)
    conv_25 = BatchNormalization()(conv_25)
    conv_25 = Activation("relu")(conv_25)

    conv_26 = Convolution2D(n_labels, (1, 1), padding="valid")(conv_25)
    conv_26 = BatchNormalization()(conv_26)
    conv_26 = Reshape(
            (input_shape[0]*input_shape[1], n_labels),
            input_shape=(input_shape[0], input_shape[1], n_labels))(conv_26)

    outputs = Activation(output_mode)(conv_26)
    print("Build decoder done..")

    model = Model(inputs=inputs, outputs=outputs, name="SegNet")

    return model


model = load_model("/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/segnet-more-entropy.h5")#segnet((512, 512, 1), 4)
model.compile(optimizer = Adam(lr = 0.0001), 
              loss = [dice_loss(smooth=1e-5,thresh=0.5)], 
              metrics=['accuracy'])
#model.load_weights('/media/dril/New Volume/LUNA/models/segnet.h5')

K.get_session().close()
K.set_session(tf.Session())
K.get_session().run(tf.global_variables_initializer())


import keras

# model    = load_model('/media/dril/New Volume/LUNA/models/unet-cnn.h5')
# model.pop()
# base_model_layers = model.output
# pred = Dense(4, activation='softmax')(base_model_layers)
# model = Model(inputs=model.input, outputs=pred)

trainids = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/TRAIN.npy')
valids   = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/VALIDATION.npy')



for i in range(10000):
    np.random.shuffle(trainids)
    for t in trainids[:5]:
        x            = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-NII/'+t+'.nii.gz'))+1024.0
        x[x > 2000]  = 2000
        x[x < -2000] = -2000
        x = x/1024.0
        
        y = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-SEG-GROUND/'+t+'.mhd')).astype('uint8')
        
        y[y == 3] = 2
        y[y == 4] = 1
        y[y == 5] = 3
        
        x = np.expand_dims(x, axis=-1)
        y = np.expand_dims(y, axis=-1)
        
        y = keras.utils.to_categorical(y, num_classes=4, dtype='uint8')
        
        #print(x.shape, y.shape)
        model.fit(x, np.reshape(y, [len(y), 512*512, 4]), batch_size=4, verbose=True)
    
    print('Evaluating the Model')
    np.random.shuffle(valids)
    for t in valids[:5]:
        #print(t)
        x = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-NII/'+t+'.nii.gz'))+1024.0
        x[x > 2000]  = 2000
        x[x < -2000] = -2000
        x = x/1024.0
        y = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-SEG-GROUND/'+t+'.mhd')).astype('uint8')
        
        y[y == 3] = 2
        y[y == 4] = 1
        y[y == 5] = 3
        
        x = np.expand_dims(x, axis=-1)
        y = np.expand_dims(y, axis=-1)
        
        y = keras.utils.to_categorical(y, num_classes=4, dtype='uint8')
        
        #print(x.shape, y.shape)        
        print(t, model.evaluate(x, np.reshape(y, [len(y), 512*512, 4]), batch_size=16))
        
    model.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/segnet-4-class-dice.h5')
    #model.save('/media/dril/New Volume/LUNA/models/segnet-more.h5')

In [ ]:
# [BLOCK 10] For training the Dilated CNN Model

import keras

model    = LN.get_model(4)
#model.load_weights('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/dilated-cnn-4-class-entropy.h5')

trainids = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/TRAIN.npy')
valids   = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/VALIDATION.npy')

for i in range(10000):
    np.random.shuffle(trainids)
    for t in trainids[:20]:
        #print(t)
        x = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-NII/'+t+'.nii.gz'))+1024.0
        x[x > 2000]  = 2000
        x[x < -2000] = -2000
        x = x/1024.0
        y = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-SEG-GROUND/'+t+'.mhd')).astype('uint8')
        
        y[y == 3] = 2
        y[y == 4] = 1
        y[y == 5] = 3
        
        x = np.expand_dims(x, axis=-1)
        y = np.expand_dims(y, axis=-1)
        
        y = keras.utils.to_categorical(y, num_classes=4, dtype='uint8')
        
        #print(x.shape, y.shape)
        model.fit(x, y, batch_size=1, verbose=True)
    
    #if i%10 == 0:
    if i%10 == 0:
        print('Evaluating the Model')
        np.random.shuffle(valids)
        for t in valids[:5]:
            #print(t)
            x = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-NII/'+t+'.nii.gz'))+1024.0
            x[x > 2000]  = 2000
            x[x < -2000] = -2000
            x = x/1024.0
            y = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-SEG-GROUND/'+t+'.mhd')).astype('uint8')

            y[y == 3] = 2
            y[y == 4] = 1
            y[y == 5] = 3

            x = np.expand_dims(x, axis=-1)
            y = np.expand_dims(y, axis=-1)

            y = keras.utils.to_categorical(y, num_classes=4, dtype='uint8')

            #print(x.shape, y.shape)        
            print(model.evaluate(x, y, batch_size=1))
        
    model.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/dilated-cnn-4-class-dice2.h5')

In [27]:
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Seed the random number generator.
# This ensures that the results below are reproducible.

np.random.seed(9999)
m1 = np.random.random(150)
m2 = np.random.random(150)

f, ax = plt.subplots(1, figsize = (8,5))
sm.graphics.mean_diff_plot(m1, m2, ax = ax)

plt.show()

In [ ]:
# For training LGAN

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import keras.backend as K
from skimage.transform import resize
import matplotlib.pyplot as plt

import sys
import keras
import numpy as np

import nibabel as nib
class WGAN():
    def myloss1(self, y_true, y_pred):
        return y_pred
    
    def myloss2(self, y_true, y_pred):
        return -y_pred
    
    def __init__(self):
        self.img_rows = 224
        self.img_cols = 224
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        # Following parameter and optimizer set as recommended in paper
        self.n_gen      = 10 
        self.n_critic   = 1
        self.clip_value = 0.001
        optimizer1       = Adam(lr=0.0001)
        optimizer2       = Adam(lr=0.000001)
        optimizer3       = Adam(lr=0.0001)
        
        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.myloss1,
            optimizer=optimizer2,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer1, metrics=['accuracy'])
        
        # The generator takes noise as input and generated imgs
        ct_image      = Input(shape=(self.img_shape))
        ground_mask   = Input(shape=(self.img_shape))
        
        predicted_mask = self.generator(ct_image)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated images as input and determines validity
        em_loss = self.critic([predicted_mask, ground_mask])

        # The combined model  (stacked generator and critic)
        self.combined = Model([ct_image, ground_mask], [predicted_mask, em_loss])
        self.combined.compile(loss=['binary_crossentropy', self.myloss2],
                              loss_weights=[0.999, 0.001], optimizer=optimizer3)
    
    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):
        filter0 = 64
        filter1 = 128
        filter2 = 256
        filter3 = 512
        filter4 = 1024

        inputs = Input((224, 224, 1))

        conv0 = Conv2D(filter0, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv0 = LeakyReLU(alpha=0.02)(conv0)
        conv0 = Conv2D(filter0, 3, padding = 'same', kernel_initializer = 'he_normal')(conv0)
        conv0 = LeakyReLU(alpha=0.02)(conv0)

        pool1 = MaxPooling2D(pool_size=(2,2))(conv0)


        conv1 = Conv2D(filter1, 3,  padding = 'same', kernel_initializer = 'he_normal')(pool1)
        conv1 = LeakyReLU(alpha=0.02)(conv1)
        conv1 = Conv2D(filter1, 3,  padding = 'same', kernel_initializer = 'he_normal')(conv1)
        conv1 = LeakyReLU(alpha=0.02)(conv1)

        pool2 = MaxPooling2D(pool_size=(2,2))(conv1)


        conv2 = Conv2D(filter2, 3,  padding = 'same', kernel_initializer = 'he_normal')(pool2)
        conv2 = LeakyReLU(alpha=0.02)(conv2)
        conv2 = Conv2D(filter2, 3,  padding = 'same', kernel_initializer = 'he_normal')(conv2)
        conv2 = LeakyReLU(alpha=0.02)(conv2)

        pool3 = MaxPooling2D(pool_size=(2,2))(conv2)


        conv3 = Conv2D(filter3, 3,  padding = 'same', kernel_initializer = 'he_normal')(pool3)
        conv3 = LeakyReLU(alpha=0.02)(conv3)
        conv3 = Conv2D(filter3, 3,  padding = 'same', kernel_initializer = 'he_normal')(conv3)
        conv3 = LeakyReLU(alpha=0.02)(conv3)
        conv3 = Dropout(0.4)(conv3)
        
        pool4 = MaxPooling2D(pool_size=(2,2))(conv3)


        conv4 = Conv2D(filter4, 3,  padding = 'same', kernel_initializer = 'he_normal')(pool4)
        conv4 = LeakyReLU(alpha=0.02)(conv4)
        conv4 = Conv2D(filter4, 3,  padding = 'same', kernel_initializer = 'he_normal')(conv4)
        conv4 = LeakyReLU(alpha=0.02)(conv4)
        conv4 = Dropout(0.4)(conv4)


        temp1 = Conv2DTranspose(filters=filter4, kernel_size = (2,2), strides=(2,2))(conv4)
        temp1 = keras.layers.Concatenate()([temp1, conv3])

        up1 = Conv2D(filter3, 3,  padding = 'same', kernel_initializer = 'he_normal')(temp1)
        up1 = LeakyReLU(alpha=0.02)(up1)
        up1 = Conv2D(filter3, 3,  padding = 'same', kernel_initializer = 'he_normal')(up1)
        up1 = LeakyReLU(alpha=0.02)(up1)


        temp2 = Conv2DTranspose(filters=filter3, kernel_size = (2,2), strides=(2,2))(up1)
        temp2 = keras.layers.Concatenate()([temp2, conv2])

        up2 = Conv2D(filter2, 3,  padding = 'same', kernel_initializer = 'he_normal')(temp2)
        up2 = LeakyReLU(alpha=0.02)(up2)
        up2 = Conv2D(filter2, 3,  padding = 'same', kernel_initializer = 'he_normal')(up2)
        up2 = LeakyReLU(alpha=0.02)(up2)


        temp3 = Conv2DTranspose(filters=filter2, kernel_size = (2,2), strides=(2,2))(up2)
        temp3 = keras.layers.Concatenate()([temp3, conv1])

        up3 = Conv2D(filter1, 3,  padding = 'same', kernel_initializer = 'he_normal')(temp3)
        up3 = LeakyReLU(alpha=0.02)(up3)
        up3 = Conv2D(filter1, 3,  padding = 'same', kernel_initializer = 'he_normal')(up3)
        up3 = LeakyReLU(alpha=0.02)(up3)


        temp4 = Conv2DTranspose(filters=filter1, kernel_size = (2,2), strides=(2,2))(up3)
        temp4 = keras.layers.Concatenate()([temp4, conv0])

        up4 = Conv2D(filter0, 3,  padding = 'same', kernel_initializer = 'he_normal')(temp4)
        up4 = LeakyReLU(alpha=0.02)(up4)
        up4 = Conv2D(filter0, 3,  padding = 'same', kernel_initializer = 'he_normal')(up4)
        up4 = LeakyReLU(alpha=0.02)(up4)


        out = Conv2D(1, 1, padding='same', activation='sigmoid', kernel_initializer = 'he_normal')(up4)

        return Model(inputs, out)
    
    def build_critic(self):
        filter0 = 64
        filter1 = 128
        filter2 = 256
        filter3 = 512
        filter4 = 512

        inputs_1 = Input((224, 224, 1))
        inputs_2 = Input((224, 224, 1))

        conv0 = Conv2D(filter0, 3, kernel_initializer = 'he_normal')(inputs_1)
        conv0 = BatchNormalization()(conv0)
        conv0 = LeakyReLU(alpha=0.02)(conv0)

        conv1 = Conv2D(filter0, 3, strides=(2,2), kernel_initializer = 'he_normal')(conv0)
        conv1 = BatchNormalization()(conv1)
        conv1 = LeakyReLU(alpha=0.02)(conv1)

        conv2 = Conv2D(filter1, 3, strides=(2,2), kernel_initializer = 'he_normal')(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = LeakyReLU(alpha=0.02)(conv2)

        conv3 = Conv2D(filter1, 3, strides=(2,2), kernel_initializer = 'he_normal')(conv2)
        conv3 = BatchNormalization()(conv3)
        conv3 = LeakyReLU(alpha=0.02)(conv3)




        conv0_2 = Conv2D(filter0, 3, kernel_initializer = 'he_normal')(inputs_2)
        conv0_2 = BatchNormalization()(conv0_2)
        conv0_2 = LeakyReLU(alpha=0.02)(conv0_2)

        conv1_2 = Conv2D(filter0, 3, strides=(2,2), kernel_initializer = 'he_normal')(conv0_2)
        conv1_2 = BatchNormalization()(conv1_2)
        conv1_2 = LeakyReLU(alpha=0.02)(conv1_2)

        conv2_2 = Conv2D(filter1, 3, strides=(2,2), kernel_initializer = 'he_normal')(conv1_2)
        conv2_2 = BatchNormalization()(conv2_2)
        conv2_2 = LeakyReLU(alpha=0.02)(conv2_2)

        conv3_2 = Conv2D(filter1, 3, strides=(2,2), kernel_initializer = 'he_normal')(conv2_2)
        conv3_2 = BatchNormalization()(conv3_2)
        conv3_2 = LeakyReLU(alpha=0.02)(conv3_2)


        temp1 = keras.layers.Concatenate()([conv3, conv3_2])


        conv4 = Conv2D(filter2, 3, strides=(2,2), kernel_initializer = 'he_normal')(temp1)
        conv4 = BatchNormalization()(conv4)
        conv4 = LeakyReLU(alpha=0.02)(conv4)

        conv5 = Conv2D(filter3, 3, strides=(2,2), kernel_initializer = 'he_normal')(conv4)
        conv5 = BatchNormalization()(conv5)
        conv5 = LeakyReLU(alpha=0.02)(conv5)

        out1 = Flatten()(conv5)
        out2 = Dense(1024, kernel_initializer = 'he_normal')(out1)
        out2 = LeakyReLU(alpha=0.02)(out2)

        out  = Dense(1, kernel_initializer = 'he_normal')(out2)

        return Model([inputs_1, inputs_2], out)

    def train(self, epochs, batch_size=8, sample_interval=50):
        
        trainids = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/TRAIN.npy')
        valids   = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/VALIDATION.npy')
        
        for epoch in range(epochs):
            np.random.shuffle(trainids)
            for t in trainids[:5]:
                print(t)
                x11           = nib.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-NII/'+t+'.nii.gz')
                
                x            = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-NII/'+t+'.nii.gz'))+1024.0
                x[x > 2000]  = 2000
                x[x < -2000] = -2000
                x = x/1024.0

                y = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-SEG-GROUND/'+t+'.mhd')).astype('uint8')

                y[y == 3] = 1
                y[y == 4] = 1
                y[y == 5] = 0

                x1 = np.expand_dims(x, axis=-1)
                y1 = np.expand_dims(y, axis=-1)
                
                x = np.zeros([len(x1), 224, 224, 1])
                y = np.zeros([len(y1), 224, 224, 1])
                
                for wk in range(len(x1)):
                    x[wk, :, :, 0] = resize(x1[wk, :, :, 0], [224, 224], order=1, preserve_range=True)
                for wk in range(len(y1)):
                    y[wk, :, :, 0] = resize(y1[wk, :, :, 0], [224, 224], order=0, preserve_range=True)
                
#                 c1 = np.zeros([512, 512, x.shape[0]])
#                 for k in range(x.shape[0]):
#                     c1[:, :, k] = resize(x[k, :, :, 0], [512, 512], order=1, preserve_range=True)
#                 result = nib.Nifti1Image(c1, x11.affine, x11.header)
#                 nib.save(result, 'x.nii.gz')
                
#                 c1 = np.zeros([512, 512, x.shape[0]])
#                 for k in range(x.shape[0]):
#                     c1[:, :, k] = resize(y[k, :, :, 0], [512, 512], order=0, preserve_range=True)
#                 result = nib.Nifti1Image(c1, x11.affine, x11.header)
#                 nib.save(result, 'y.nii.gz')
                
                #print(np.max(y.flatten()), np.min(y.flatten()), np.max(x.flatten()), np.min(x.flatten()))
                #if epoch < 5:
                ###    self.generator.fit(x, y, batch_size=8, epochs=5)
                
                result = self.generator.predict(x, batch_size=8)
                
                for l in self.critic.layers:
                    l.trainable = True
                
                for pk in range(self.n_critic):
                    # Train the critic
                    c_loss = self.critic.fit([result, y], np.zeros([result.shape[0], 1]), batch_size=16, epochs=1, verbose=False)
                    
#                     # Clip critic weights
#                     for l in self.critic.layers:
#                         weights = l.get_weights()
#                         weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
#                         l.set_weights(weights)
                    
                    #if pk == 4:
                    print(epoch, ' c_loss ', c_loss.history['loss'])
    
                # ---------------------
                #  Train Combined Generator
                # ---------------------
                
#                 countc = 0
#                 while(True):
#                     g_loss = self.combined.fit([x, y], [y, np.zeros([result.shape[0], 1])], batch_size=4, epochs=1, verbose=False)
#                     k_list = list(g_loss.history.keys())
#                     countc = countc+1
#                     if g_loss.history[k_list[1]][-1] < 0.002 or countc > 10:
#                         break
                for l in self.critic.layers:
                    l.trainable = False
                
                g_loss = self.combined.fit([x, y], [y, np.zeros([x.shape[0], 1])], batch_size=8, epochs=5, verbose=False)
                k_list = list(g_loss.history.keys())
                print(epoch, ' g_loss ', g_loss.history[k_list[0]][-1], g_loss.history[k_list[1]][-1], g_loss.history[k_list[2]][-1])
                
                
                result[result > 0.5] = 1
                result[result < 0.5] = 0
                result = result.astype('uint8')

                c1 = np.zeros([512, 512, result.shape[0]])
                for k in range(result.shape[0]):
                    c1[:, :, k] = resize(result[k, :, :, 0], [512, 512], order=0, preserve_range=True)

                c1     = c1.astype('uint8')
                result = nib.Nifti1Image(c1, x11.affine, x11.header)
                nib.save(result, 'test.nii.gz')
            
            if epoch %5 == 0:
                self.critic.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/lgan_critic1.h5')
                self.generator.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/lgan_generator1.h5')
                
            
        
wgan = WGAN()
wgan.train(epochs=4000, batch_size=4, sample_interval=50)

In [4]:
# For training Fully convolutional Multi instance GAN

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import keras.backend as K
from skimage.transform import resize
import matplotlib.pyplot as plt

import sys
import keras
import numpy as np

import nibabel as nib

import keras.backend as K

from keras import regularizers
from keras.initializers import RandomNormal

def channelPool_max(x):
    return K.max(x,axis=-1)

def channelPool_avg(x):
    return K.mean(x,axis=-1)


class WGAN():
    
    def myloss_l1(self, y_true, y_pred):
        mask = (y_true-0.5)*2    
        out = -Multiply()([mask, y_pred])
        out = 1+K.exp(out)
        out = K.log(out)
        out = Multiply()([out, y_true])
        #print(out.shape)
        #out = Reshape((-1, ))
        #out = Flatten()(out)
        out = K.mean(out, axis=[1, 2])
        print(out.shape)
        return out
    
    def myloss_l2(self, y_true, y_pred):
        mask = (y_true-0.5)*2
        out = -Multiply()([mask, y_pred])
        out = 1+K.exp(out)
        out = K.log(out)
        out = Multiply()([out, 1-y_true])
        #print(out.shape)
        #out = Flatten()(out)
        out = K.mean(out, axis=[1, 2])
        print(out.shape)
        return out
    
    def __init__(self):
        self.img_rows = 512
        self.img_cols = 512
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        # Following parameter and optimizer set as recommended in paper
        self.n_gen      = 10 
        self.n_critic   = 1
        self.clip_value = 0.001
        optimizer1       = Adam(lr=0.0001)
        optimizer2       = Adam(lr=0.0001)
        optimizer3       = Adam(lr=0.0001)
        
        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss='binary_crossentropy',
            optimizer=optimizer1,
            metrics=['accuracy'])
        self.critic.load_weights('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/lgan_critic_multi4.h5')
        
        # Build the generator
        self.generator = self.build_generator()
        self.generator.compile(loss=['binary_crossentropy', self.myloss_l1, self.myloss_l2],
                              loss_weights=[1, 1, 10], optimizer=optimizer1)
        print(self.generator.summary())
        self.generator.load_weights('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/lgan_generator_multi3-withgan.h5')
        
        #he generator takes noise as input and generated imgs
        ct_image      = Input(shape=(self.img_shape))
        
        #[predicted_mask, mil1_l1, mil1_l2, mil2_l1, mil2_l2] = self.generator(ct_image)
        [predicted_mask, mil2_l1, mil2_l2] = self.generator(ct_image)

        #For the combined model we will only train the generator
        self.critic.trainable = False

        #The critic takes generated images as input and determines validity
        validity = self.critic(predicted_mask)

#         #The combined model  (stacked generator and critic)
#         #self.combined = Model(ct_image, [validity, predicted_mask, mil1_l1, mil1_l2, mil2_l1, mil2_l2])
        self.combined = Model(ct_image, [validity, predicted_mask, mil2_l1, mil2_l2])
        self.combined.compile(loss=['binary_crossentropy', 'binary_crossentropy', self.myloss_l1, self.myloss_l2],
                          loss_weights=[1, 100, 1, 10], optimizer=optimizer1)
        
        #self.combined.compile(loss=['binary_crossentropy', 'binary_crossentropy', self.myloss_l1, self.myloss_l1],
        #                    loss_weights=[1, 1, 1, 10], optimizer=optimizer3)
        
        print(self.critic.summary())
    
    def build_generator(self):
        filter0 = 48
        filter1 = 128
        filter2 = 256
        filter3 = 512

        inputs = Input((512, 512, 1))
        
        #kernel_regularizer=regularizers.l2(0.01),
        # Downsampling Code
        conv0 = Conv2D(filter0, 8, padding = 'same', activation='relu',   kernel_initializer = RandomNormal(mean=0.0, stddev=0.05, seed=1))(inputs)
        pool0 = MaxPooling2D(pool_size=(2,2))(conv0)

        conv1 = Conv2D(filter1, 2, padding = 'same', activation='relu',   kernel_initializer = RandomNormal(mean=0.0, stddev=0.05, seed=1))(pool0)
        pool1 = MaxPooling2D(pool_size=(2,2))(conv1)

        #mil1_l1 = Lambda(channelPool_max)(pool1)
        #mil1_l2 = Lambda(channelPool_avg)(pool1)
        
        #p1    = Reshape((128, 128, 1))(mil1_l1)
        #p2    = Reshape((128, 128, 1))(mil1_l2)
        #temp1 = Concatenate()([p1, p2, pool1])
        
        conv2 = Conv2D(filter2, 2, padding = 'same', activation='relu',   kernel_initializer = RandomNormal(mean=0.0, stddev=0.05, seed=1))(pool1)
        pool2 = MaxPooling2D(pool_size=(2,2))(conv2)

        mil2_l1 = Lambda(channelPool_max)(pool2)
        mil2_l2 = Lambda(channelPool_avg)(pool2)
        
        p3    = Reshape((64, 64, 1))(mil2_l1)
        p4    = Reshape((64, 64, 1))(mil2_l2)
        temp2 = Concatenate()([p3, p4, pool2])
        
        conv3 = Conv2D(filter3, 2, padding = 'same', activation='relu',  kernel_initializer = RandomNormal(mean=0.0, stddev=0.05, seed=1))(temp2)
        pool3 = MaxPooling2D(pool_size=(2,2))(conv3)

        
        
        # UpSampling Code
        conv4  = Conv2DTranspose(filters=filter2,  activation='relu', kernel_size = (2,2),  strides=(2,2))(pool3)
        merge1 = Concatenate()([conv4, pool2])
        conv4  = Conv2D(filter2, 2, padding = 'same', activation='relu',  kernel_initializer = RandomNormal(mean=0.0, stddev=0.05, seed=1))(merge1)

        conv5  = Conv2DTranspose(filters=filter1,  activation='relu',  kernel_size = (2,2),  strides=(2,2))(conv4)
        merge2 = Concatenate()([conv5, pool1])
        conv5  = Conv2D(filter1, 2, padding = 'same', activation='relu',   kernel_initializer = RandomNormal(mean=0.0, stddev=0.05, seed=1))(merge2)

        conv6  = Conv2DTranspose(filters=filter0,  activation='relu', kernel_size = (2,2),   strides=(2,2))(conv5)
        merge3 = Concatenate()([conv6, pool0])
        conv6  = Conv2D(filter0, 2, padding = 'same', activation='relu',   kernel_initializer = RandomNormal(mean=0.0, stddev=0.05, seed=1))(merge3)

        conv7  = Conv2DTranspose(filters=filter0,  activation='relu',  kernel_initializer = RandomNormal(mean=0.0, stddev=0.05, seed=1), kernel_size = (2,2), strides=(2,2))(conv6)

        out = Conv2D(1, 1, padding='same', activation='sigmoid')(conv7)

        return Model(inputs, [out, mil2_l1, mil2_l2])
    
    def build_critic(self):
        filter0 = 48
        filter1 = 128
        filter2 = 256
        filter3 = 512

        inputs = Input((512, 512, 1))
        
        #inputs_1 = Lambda(lambda x: x[:, :, :, 1])(inputs)
        #inputs_1 = Reshape((512, 512, 1))(inputs_1)
        
        conv0 = Conv2D(filter0, 8, activation='relu', kernel_initializer = 'he_normal')(inputs)
        pool0 = MaxPooling2D(pool_size=(2,2))(conv0)

        conv1 = Conv2D(filter1, 2, activation='relu', kernel_initializer = 'he_normal')(pool0)
        pool1 = MaxPooling2D(pool_size=(2,2))(conv1)

        conv2 = Conv2D(filter2, 2, activation='relu', kernel_initializer = 'he_normal')(pool1)
        pool2 = MaxPooling2D(pool_size=(2,2))(conv2)

        conv3 = Conv2D(filter3, 2, activation='relu', kernel_initializer = 'he_normal')(pool2)
        pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
        #pool3 = Activation('sigmoid')(pool3)
        
        out   = Conv2D(1, 1, activation='sigmoid', kernel_initializer = 'he_normal')(pool3)
        #out   = Dense(1, activation='sigmoid', kernel_initializer = 'he_normal')(conv4)
        
        #conv4 = Flatten()(conv4)
        #

        return Model(inputs, out)

    def train(self, epochs, batch_size=8, sample_interval=50):
        
        trainids = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/TRAIN.npy')
        valids   = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/VALIDATION.npy')
        
        allx  = []
        ally  = []
        ally2 = []
        
        for epoch in range(epochs):
            np.random.shuffle(trainids)
            
            allx  = []
            ally  = []
            ally2 = []
        
            for t in trainids[:5]:
                print(t)
                x11           = nib.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-NII/'+t+'.nii.gz')
                
                x            = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-NII/'+t+'.nii.gz'))+1024.0
                x[x > 2000]  = 2000
                x[x < -2000] = -2000
                x = x/1024.0

                y = sitk.GetArrayFromImage(sitk.ReadImage('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/ORIG-SEG-GROUND/'+t+'.mhd')).astype('uint8')

                y[y == 3] = 1
                y[y == 4] = 1
                y[y == 5] = 0

                x = np.expand_dims(x, axis=-1)
                y = np.expand_dims(y, axis=-1)
                
#                 y1 = np.zeros([len(y), 128, 128])
#                 for wk in range(len(y1)):
#                     y1[wk, :, :] = resize(y[wk, :, :, 0], [128, 128], order=0, preserve_range=True)
                
                y2 = np.zeros([len(y), 64, 64])
                for wk in range(len(y2)):
                    y2[wk, :, :] = resize(y[wk, :, :, 0], [64, 64], order=0, preserve_range=True)
                
                x = x[::5]
                y = y[::5]
                y2 = y2[::5]
                
                allx.append(x)
                ally.append(y)
                ally2.append(y2)
                #print('All shapes are ', x.shape, y.shape, y2.shape)
            
            x  = np.concatenate(allx)
            y  = np.concatenate(ally)
            y2 = np.concatenate(ally2)
            print('All shapes are Final ', x.shape, y.shape, y2.shape)
            
            #y_cat_2 = keras.utils.to_categorical(y, num_classes=2, dtype='uint8')

            #g_loss = self.generator.fit(x, [y, y2, y2], batch_size=8, epochs=5, verbose=True)
            #k_list = list(g_loss.history.keys())
            #print(k_list)
            #print(epoch, ' g_loss ', g_loss.history[k_list[0]][-1], g_loss.history[k_list[1]][-1], g_loss.history[k_list[2]][-1])

            result, _, _ = self.generator.predict(x, batch_size=8)
            #result = result[:, :, :, 1]
            #result = np.expand_dims(result axis=-1)

            #for l in self.critic.layers:
            #    l.trainable = True

            for pk in range(self.n_critic):
                # Train the critic
                c_loss1 = self.critic.fit(result, np.zeros([result.shape[0], 30, 30, 1]), batch_size=8, epochs=5, verbose=False)
                c_loss2 = self.critic.fit(y,      np.ones([result.shape[0], 30, 30, 1]),  batch_size=8, epochs=5, verbose=False)
            print(epoch, ' c_loss1 ', c_loss1.history['loss'], ' c_loss2 ', c_loss2.history['loss'])

            #for l in self.critic.layers:
            #    l.trainable = False

            g_loss = self.combined.fit(x, [np.ones([x.shape[0], 30, 30, 1]), y, y2, y2], batch_size=4, epochs=5, verbose=False)
            k_list = list(g_loss.history.keys())
            #print(k_list)
            print(epoch, ' g_loss ', g_loss.history[k_list[0]][-1], g_loss.history[k_list[1]][-1], g_loss.history[k_list[2]][-1])

#                 #result[result > 0.5] = 1
#                 #result[result < 0.5] = 0
#                 #result = result.astype('uint8')

            result[result > 0.5] = 1
            result[result < 0.5] = 0
            #result  = np.argmax(result, axis=-1)
            #y_cat_2 = np.argmax(y_cat_2, axis=-1)

            c1     = np.zeros([512, 512, result.shape[0]])
            for k in range(result.shape[0]):
                c1[:, :, k] = resize(result[k, :, :, 0], [512, 512], order=0, preserve_range=True)
            result = nib.Nifti1Image(c1, x11.affine, x11.header)
            nib.save(result, 'test.nii.gz')

#                 c1     = np.zeros([512, 512, y_cat_2.shape[0]])
#                 for k in range(y_cat_2.shape[0]):
#                     c1[:, :, k] = resize(y_cat_2[k, :, :], [512, 512], order=0, preserve_range=True)
#                 result = nib.Nifti1Image(c1, x11.affine, x11.header)
#                 nib.save(result, 'testr.nii.gz')

#                 c2     = np.zeros([512, 512, x.shape[0]])
#                 for k in range(x.shape[0]):
#                     c2[:, :, k] = resize(x[k, :, :, 0], [512, 512], order=0, preserve_range=True)
#                 result = nib.Nifti1Image(c2, x11.affine, x11.header)
#                 nib.save(result, 'testx.nii.gz')

            if epoch %5 == 0:
                self.critic.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/lgan_critic_multi4.h5')
                self.generator.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/LUNA/models/lgan_generator_multi3-withgan.h5')

            
        
wgan = WGAN()
wgan.train(epochs=4000, batch_size=4, sample_interval=50)

(?,)
(?,)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 512, 512, 48) 3120        input_10[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_17 (MaxPooling2D) (None, 256, 256, 48) 0           conv2d_48[0][0]                  
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, 256, 256, 128 24704       max_pooling2d_17[0][0]           
__________________________________________________________________________________________________


0  g_loss  10.039798496005771 0.8059954532273896 0.6352454976992564
1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674661221381920536987
1.3.6.1.4.1.14519.5.2.1.6279.6001.133132722052053001903031735878
1.3.6.1.4.1.14519.5.2.1.6279.6001.232071262560365924176679652948
1.3.6.1.4.1.14519.5.2.1.6279.6001.205852555362702089950453265567
1.3.6.1.4.1.14519.5.2.1.6279.6001.655242448149322898770987310561
All shapes are Final  (316, 512, 512, 1) (316, 512, 512, 1) (316, 64, 64)
1  c_loss1  [1.4170069502124303, 0.09076010657450821, 0.030424758226056642, 0.014432895598532279, 0.006992453514727988]  c_loss2  [1.770159126082553, 0.16221849639204483, 0.06826270890386799, 0.04082303263152702, 0.028752422078123577]
1  g_loss  10.04375669624232 0.9027067739752275 0.6514199564728556
1.3.6.1.4.1.14519.5.2.1.6279.6001.553241901808946577644850294647
1.3.6.1.4.1.14519.5.2.1.6279.6001.897684031374557757145405000951
1.3.6.1.4.1.14519.5.2.1.6279.6001.438308540025607517017949816111
1.3.6.1.4.1.14519.5.2.1.6279.6001.12

12  g_loss  10.260921562774271 2.9695098445385315 0.6411743767653839
1.3.6.1.4.1.14519.5.2.1.6279.6001.413896555982844732694353377538
1.3.6.1.4.1.14519.5.2.1.6279.6001.162901839201654862079549658100
1.3.6.1.4.1.14519.5.2.1.6279.6001.209269973797560820442292189762
1.3.6.1.4.1.14519.5.2.1.6279.6001.811825890493256320617655474043
1.3.6.1.4.1.14519.5.2.1.6279.6001.140527383975300992150799777603
All shapes are Final  (205, 512, 512, 1) (205, 512, 512, 1) (205, 64, 64)
13  c_loss1  [1.795039160077165, 0.9778323598024322, 0.6236412138473697, 0.4600510495464976, 0.31397353323494515]  c_loss2  [1.0027372537589654, 0.578661251649624, 0.44039093663052814, 0.37631901095553144, 0.33582236621438005]
13  g_loss  10.35300021985682 3.0223832627622094 0.6524698414453646
1.3.6.1.4.1.14519.5.2.1.6279.6001.189483585244687808087477024767
1.3.6.1.4.1.14519.5.2.1.6279.6001.267957701183569638795986183786
1.3.6.1.4.1.14519.5.2.1.6279.6001.190144948425835566841437565646
1.3.6.1.4.1.14519.5.2.1.6279.6001.19448853

24  g_loss  10.32627562871055 2.726498163314093 0.6499416209402539
1.3.6.1.4.1.14519.5.2.1.6279.6001.114195693932194925962391697338
1.3.6.1.4.1.14519.5.2.1.6279.6001.126704785377921920210612476953
1.3.6.1.4.1.14519.5.2.1.6279.6001.202187810895588720702176009630
1.3.6.1.4.1.14519.5.2.1.6279.6001.310395752124284049604069960014
1.3.6.1.4.1.14519.5.2.1.6279.6001.265453131727473342790950829556
All shapes are Final  (248, 512, 512, 1) (248, 512, 512, 1) (248, 64, 64)
25  c_loss1  [1.472938713527495, 0.20987371138988004, 0.06283424001547598, 0.02711542353274361, 0.014217934240737268]  c_loss2  [1.756376024215452, 0.4383939552691675, 0.19330350477849284, 0.10395838176050494, 0.0607712589684994]
25  g_loss  10.604897037629158 3.075975100840292 0.631063844888441
1.3.6.1.4.1.14519.5.2.1.6279.6001.215086589927307766627151367533
1.3.6.1.4.1.14519.5.2.1.6279.6001.142154819868944114554521645782
1.3.6.1.4.1.14519.5.2.1.6279.6001.308183340111270052562662456038
1.3.6.1.4.1.14519.5.2.1.6279.6001.20926997

36  g_loss  10.998279079794884 0.7372936164028943 0.6283507850021124
1.3.6.1.4.1.14519.5.2.1.6279.6001.143412474064515942785157561636
1.3.6.1.4.1.14519.5.2.1.6279.6001.250397690690072950000431855143
1.3.6.1.4.1.14519.5.2.1.6279.6001.134370886216012873213579659366
1.3.6.1.4.1.14519.5.2.1.6279.6001.898642529028521482602829374444
1.3.6.1.4.1.14519.5.2.1.6279.6001.885168397833922082085837240429
All shapes are Final  (273, 512, 512, 1) (273, 512, 512, 1) (273, 64, 64)
37  c_loss1  [1.597036707641441, 0.27355900477795375, 0.17619403579951207, 0.12526342897733925, 0.09387681150174403]  c_loss2  [1.596895293875055, 0.6776492191758348, 0.3811605284502218, 0.22960538561745877, 0.1472948765580034]
37  g_loss  9.819904578911078 0.7697756388685205 0.6498642924067738
1.3.6.1.4.1.14519.5.2.1.6279.6001.779493719385047675154892222907
1.3.6.1.4.1.14519.5.2.1.6279.6001.184019785706727365023450012318
1.3.6.1.4.1.14519.5.2.1.6279.6001.294188507421106424248264912111
1.3.6.1.4.1.14519.5.2.1.6279.6001.2048022

...

396  g_loss  10.980717045920235 3.5410805480820793 0.6396410184247153
1.3.6.1.4.1.14519.5.2.1.6279.6001.244681063194071446501270815660
1.3.6.1.4.1.14519.5.2.1.6279.6001.238042459915048190592571019348
1.3.6.1.4.1.14519.5.2.1.6279.6001.177685820605315926524514718990
1.3.6.1.4.1.14519.5.2.1.6279.6001.334184846571549530235084187602
1.3.6.1.4.1.14519.5.2.1.6279.6001.176638348958425792989125209419
All shapes are Final  (214, 512, 512, 1) (214, 512, 512, 1) (214, 64, 64)
397  c_loss1  [0.8719242443548185, 0.4440519741483938, 0.07272408354296306, 0.010229896714858642, 0.006034238124701464]  c_loss2  [1.5772465034344485, 0.10892163502557256, 0.047295568563113705, 0.023922273697697113, 0.014242087978277808]
397  g_loss  10.051720824197075 2.660095132399942 0.6543207948452958
1.3.6.1.4.1.14519.5.2.1.6279.6001.208737629504245244513001631764
1.3.6.1.4.1.14519.5.2.1.6279.6001.333145094436144085379032922488
1.3.6.1.4.1.14519.5.2.1.6279.6001.217589936421986638139451480826
1.3.6.1.4.1.14519.5.2.1.6279.

408  g_loss  9.403868287185142 0.37984674655157946 0.6469300315297883
1.3.6.1.4.1.14519.5.2.1.6279.6001.438308540025607517017949816111
1.3.6.1.4.1.14519.5.2.1.6279.6001.395623571499047043765181005112
1.3.6.1.4.1.14519.5.2.1.6279.6001.295420274214095686326263147663
1.3.6.1.4.1.14519.5.2.1.6279.6001.270215889102603268207599305185
1.3.6.1.4.1.14519.5.2.1.6279.6001.745109871503276594185453478952
All shapes are Final  (244, 512, 512, 1) (244, 512, 512, 1) (244, 64, 64)
409  c_loss1  [1.7676005304836837, 0.558763922237959, 0.3512868969166865, 0.16962065828628228, 0.07741778221775274]  c_loss2  [2.233629915557924, 0.5475607025818746, 0.4259316481527735, 0.3957361725510144, 0.3749017710568475]
409  g_loss  12.55470013227619 1.3439726174854842 0.6451467431959559
1.3.6.1.4.1.14519.5.2.1.6279.6001.126264578931778258890371755354
1.3.6.1.4.1.14519.5.2.1.6279.6001.270152671889301412052226973069
1.3.6.1.4.1.14519.5.2.1.6279.6001.466284753932369813717081722101
1.3.6.1.4.1.14519.5.2.1.6279.6001.2123464

420  g_loss  10.83678705236885 3.344530623950315 0.646440906471081
1.3.6.1.4.1.14519.5.2.1.6279.6001.449254134266555649028108149727
1.3.6.1.4.1.14519.5.2.1.6279.6001.216526102138308489357443843021
1.3.6.1.4.1.14519.5.2.1.6279.6001.802595762867498341201607992711
1.3.6.1.4.1.14519.5.2.1.6279.6001.184019785706727365023450012318
1.3.6.1.4.1.14519.5.2.1.6279.6001.128023902651233986592378348912
All shapes are Final  (143, 512, 512, 1) (143, 512, 512, 1) (143, 64, 64)
421  c_loss1  [1.19124066287821, 0.7395367084683239, 0.41632846039491933, 0.2646389279540602, 0.19052894371789653]  c_loss2  [1.2832234185058753, 0.7704969056836375, 0.43777135848165394, 0.2961354225457131, 0.23425793199689238]
421  g_loss  11.95493255962025 3.5947052932285763 0.650190344640425
1.3.6.1.4.1.14519.5.2.1.6279.6001.246589849815292078281051154201
1.3.6.1.4.1.14519.5.2.1.6279.6001.216882370221919561230873289517
1.3.6.1.4.1.14519.5.2.1.6279.6001.826829446346820089862659555750
1.3.6.1.4.1.14519.5.2.1.6279.6001.160586340

432  g_loss  10.041248961762115 2.24203104989512 0.6420732786605409
1.3.6.1.4.1.14519.5.2.1.6279.6001.122914038048856168343065566972
1.3.6.1.4.1.14519.5.2.1.6279.6001.898642529028521482602829374444
1.3.6.1.4.1.14519.5.2.1.6279.6001.283569726884265181140892667131
1.3.6.1.4.1.14519.5.2.1.6279.6001.300246184547502297539521283806
1.3.6.1.4.1.14519.5.2.1.6279.6001.304676828064484590312919543151
All shapes are Final  (171, 512, 512, 1) (171, 512, 512, 1) (171, 64, 64)
433  c_loss1  [2.357458832319717, 0.6862101167963263, 0.5298638366467772, 0.3640356762715948, 0.22748721909453298]  c_loss2  [1.2508098754966468, 0.39498821162340936, 0.1405002604189672, 0.06845656926171821, 0.04152725810152397]
433  g_loss  10.197096456561173 2.4113789315809284 0.6570293146964402
1.3.6.1.4.1.14519.5.2.1.6279.6001.194488534645348916700259325236
1.3.6.1.4.1.14519.5.2.1.6279.6001.313283554967554803238484128406
1.3.6.1.4.1.14519.5.2.1.6279.6001.273525289046256012743471155680
1.3.6.1.4.1.14519.5.2.1.6279.6001.11419

KeyboardInterrupt: 